In [ ]:
import os

# IMPORTANT: Fix for PyTorch/IPython compatibility issue
# This must run BEFORE importing torch to avoid decorator conflicts
# This fixes the "disable() got an unexpected keyword argument 'wrapping'" error

# Method 1: Try to disable dynamo via environment variable (needs to be set before import)
os.environ.setdefault('TORCH_COMPILE_DISABLE', '1')
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch

# Method 2: Patch torch._dynamo.disable decorator after import
try:
    import torch._dynamo
    # Patch the disable function to ignore the 'wrapping' parameter
    if hasattr(torch._dynamo, 'disable'):
        def patched_disable(fn=None, *args, **kwargs):
            # Remove problematic 'wrapping' parameter if present
            if 'wrapping' in kwargs:
                kwargs.pop('wrapping')
            if fn is None:
                # Decorator usage: @disable
                return lambda f: f
            # Function usage: disable(fn) or disable(fn, **kwargs)
            # Simply return the function unwrapped to avoid recursion
            # The original disable was causing issues, so we bypass it entirely
            return fn
        torch._dynamo.disable = patched_disable
except Exception as e:
    print(f"Warning: Could not patch torch._dynamo: {e}")
    pass  # If patching fails, continue anyway

import random, string

from torchtext import data , datasets
from collections import defaultdict, Counter
import numpy as np

os.environ['GENSIM_DATA_DIR'] = os.path.join(os.getcwd(), 'gensim-data')

import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_model

from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import time, copy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

In [ ]:
### Part 0: Dataset Preparation

# For tokenization
TEXT = data.Field ( tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True )

# For multi - class classification labels
LABEL = data.LabelField ()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Load the TREC dataset
# Train / Validation / Test split
train_data, test_data = datasets.TREC.splits( TEXT, LABEL, fine_grained = False )

train_data, validation_data = train_data.split(
    split_ratio=0.8,
    stratified=True,
    strata_field='label'
    random_state= random.seed(42)
)
print(vars(train_data.examples[0]))


# Count how many samples per label in the train set
label_counts = Counter([ex.label for ex in train_data.examples])
total_examples = len(train_data)

print("\nLabel distribution in training set:")
for label, count in sorted(label_counts.items()):
    percentage = (count / total_examples) * 100
    print(f"- {label}: {count} samples ({percentage:.2f}%)")

# Optional sanity check: total percentages should sum ≈ 100%
total_percentage = sum((count / total_examples) * 100 for count in label_counts.values())
print(f"Total samples: {total_examples}, Sum of percentages: {total_percentage:.2f}%")


In [ ]:
for ex in train_data:
    print(f'{ex.label}: {ex.text}')

# Part 1: Prepare Word Embeddings

In [ ]:
#### a) Size of Vocabulary formed from training data according to tokenization method
# Vocabulary size (includes specials like <unk>, <pad>)
TEXT.build_vocab(train_data, min_freq=1)
vocab_size = len(TEXT.vocab)
print("Vocabulary Size (with specials):", vocab_size)

vocab_wo_specials = len([w for w in TEXT.vocab.stoi if w not in {TEXT.unk_token, TEXT.pad_token}])
print("Vocabulary size (no specials):", vocab_wo_specials)

In [ ]:
#### b) How many OOV words exist in your training data?
####    What is the number of OOV words for each topic category?
# Load Word2Vec model from local file instead of downloading
w2v = api.load('word2vec-google-news-300')
# w2v = KeyedVectors.load('word2vec-google-news-300.model')
w2v_vocab = w2v.key_to_index

# Get training vocab tokens (types), excluding specials
specials = {TEXT.unk_token, TEXT.pad_token}
train_vocab_types = [w for w in TEXT.vocab.stoi.keys() if w not in specials]

# Overall OOV types in training vocab
oov_types_overall = {w for w in train_vocab_types if w not in w2v_vocab}
print("Number of OOV word types (overall):", len(oov_types_overall))

# OOV types per label (unique types per category across its sentences)
label_to_oov_types = defaultdict(set)
label_to_total_types = defaultdict(set)

for ex in train_data.examples:
    label = ex.label
    # Count by unique types per sentence to avoid overcounting repeats
    for w in set(ex.text):
        label_to_total_types[label].add(w)
        if w not in specials and w not in w2v_vocab:
            label_to_oov_types[label].add(w)

print("\nOOV word types per topic label:")
for label in sorted(label_to_total_types.keys()):
    num_oov = len(label_to_oov_types[label])
    num_types = len(label_to_total_types[label])
    rate = (num_oov / num_types) if num_types > 0 else 0.0
    print(f"- {label}: {num_oov} OOV types (out of {num_types}, rate={rate:.2%})")

In [ ]:
# #### c) OOV mitigation strategy (No transformer-based language models allowed)
# Implement your solution in your source code. Show the corresponding code snippet.
# 1. Fast Text Model Implementatation
# Load FastText with subword info (pretrained on Wikipedia)
# First download is large; cached afterwards

# 2. Modelling Unknown (<UNK>) token approach
# Make the <unk> vector informative and trainable by initializing it
# as the mean of available pretrained vectors.

# Loading fasttext model
fatter_fasttext_bin = load_facebook_model('crawl-300d-2M-subword/crawl-300d-2M-subword.bin')
embedding_dim = fatter_fasttext_bin.wv.vector_size

# Build embedding matrix aligned to TEXT.vocab
num_tokens = len(TEXT.vocab)
emb_matrix = np.zeros((num_tokens, embedding_dim), dtype=np.float32)

# torchtext 0.4.0: TEXT.vocab.itos is index->token, stoi is token->index
pad_tok = TEXT.pad_token
unk_tok = TEXT.unk_token

# Getting index of <unk> in vocab
unk_index = TEXT.vocab.stoi[TEXT.unk_token]
known_vecs = []

for idx, token in enumerate(TEXT.vocab.itos):
    # Skip specials here; we will set them explicitly below
    if token in {pad_tok, unk_tok}:
        continue

    vec = fatter_fasttext_bin.wv[token]
    emb_matrix[idx] = vec
    known_vecs.append(vec)

if len(known_vecs) > 0:
    unk_mean = torch.tensor(np.mean(known_vecs, axis=0), dtype=torch.float32)
else:
    unk_mean = torch.empty(embedding_dim).uniform_(-0.05, 0.05)
with torch.no_grad():
    emb_matrix[unk_index] = unk_mean

# Create Embedding layer initialized with FastText
fatter_embedding = torch.nn.Embedding(num_tokens, embedding_dim, padding_idx=TEXT.vocab.stoi[TEXT.pad_token])
fatter_embedding.weight.data.copy_(torch.from_numpy(emb_matrix))

torch.save(fatter_embedding, 'embedding_weights_fatter_fasttext.pt')

In [ ]:
#### d) Select the 20 most frequent words from each topic category in the training set (removing
# stopwords if necessary). Retrieve their pretrained embeddings (from Word2Vec or GloVe).
# Project these embeddings into 2D space (using e.g., t-SNE or Principal Component Analysis).
# Plot the points in a scatter plot, color-coded by their topic category. Attach your plot here.
# Analyze your findings.

# Build per-label token frequency (lowercased, stopwords/punct filtered)
label_to_counter = defaultdict(Counter)
valid_chars = set(string.ascii_letters)

def is_valid_token(tok: str) -> bool:
    t = tok.strip("'\"")
    if len(t) == 0:
        return False
    # Keep purely alphabetic tokens to avoid punctuation/numbers
    return t.isalpha()

for ex in train_data.examples:
    label = ex.label
    for tok in ex.text:
        tok_l = tok.lower()
        if tok_l in STOP_WORDS:
            continue
        if not is_valid_token(tok_l):
            continue
        label_to_counter[label][tok_l] += 1

# Select top 20 per label that exist in Word2Vec
topk = 20
label_to_top_tokens = {}
for label, ctr in label_to_counter.items():
    selected = []
    for tok, _ in ctr.most_common():
        if tok in w2v.key_to_index:
            selected.append(tok)
        if len(selected) >= topk:
            break
    label_to_top_tokens[label] = selected

# Collect embeddings and labels
points = []
point_labels = []
point_words = []
for label, toks in label_to_top_tokens.items():
    for tok in toks:
        vec = w2v.get_vector(tok)
        points.append(vec)
        point_labels.append(label)
        point_words.append(tok)

if len(points) > 0:
    X = np.vstack(points)

    # 2D projections
    # tsne_2d = TSNE(n_components=2, random_state=42, init="pca", perplexity=30).fit_transform(X)
    # pca_2d = PCA(n_components=2, random_state=42).fit_transform(X)
    tsne_2d = TSNE(n_components=2, random_state=SEED, init="pca", perplexity=30).fit_transform(X)
    pca_2d = PCA(n_components=2, random_state=SEED).fit_transform(X)

    # Assign colors per label
    unique_labels = sorted(set(point_labels))
    color_map = {lab: plt.cm.tab10(i % 10) for i, lab in enumerate(unique_labels)}

    def plot_scatter(Y2, title: str, fname: str):
        plt.figure(figsize=(10, 8))
        for lab in unique_labels:
            idxs = [i for i, l in enumerate(point_labels) if l == lab]
            plt.scatter(Y2[idxs, 0], Y2[idxs, 1], c=[color_map[lab]], label=lab, alpha=0.8, s=40)
            # Light word annotations (optional; can clutter)
            for i in idxs:
                plt.annotate(point_words[i], (Y2[i, 0], Y2[i, 1]), fontsize=7, alpha=0.7)
        plt.legend(title="TREC label")
        plt.title(title)
        plt.tight_layout()
        plt.savefig(fname, dpi=200)
        plt.close()

    plot_scatter(tsne_2d, "Top-20 per TREC label (Word2Vec) - t-SNE", "trec_top20_tsne.png")
    plot_scatter(pca_2d, "Top-20 per TREC label (Word2Vec) - PCA", "trec_top20_pca.png")

    print("Saved plots: trec_top20_tsne.png, trec_top20_pca.png")
    for lab in unique_labels:
        print(f"{lab}: {label_to_top_tokens[lab]}")
else:
    print("No points collected for visualization. Check filtering or embedding availability.")

In [ ]:
fasttext = torch.load('embedding_weights_fatter_fasttext.pt', weights_only=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
### Part 2: Model Training & Evaluation - RNN

# Build vocabulary for labels
LABEL.build_vocab(train_data)
num_classes = len(LABEL.vocab)
print(f"\nNumber of classes: {num_classes}")
print(f"Classes: {LABEL.vocab.itos}")

# Create iterators for batching (inline for easier debugging)
# train_iterator = data.BucketIterator(...)
# val_iterator = data.BucketIterator(...)
# test_iterator = data.BucketIterator(...)
# (Used directly in Part 2 execution below)


class SimpleRNNClassifier(nn.Module):
    """
    Simple RNN for topic classification (Baseline - no dropout).
    Uses pretrained embeddings (learnable/updated during training) with OOV mitigation 
    and aggregates word representations to sentence representation using the last hidden state.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=1, dropout=0.0, padding_idx=0, pretrained_embeddings=None):
        super(SimpleRNNClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # IMPORTANT: Make embeddings learnable (updated during training)
            # This allows fine-tuning of embeddings including OOV words handled by FastText
            self.embedding.weight.requires_grad = True
        
        # Simple RNN layer (no dropout in baseline)
        self.rnn = nn.RNN(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.0  # No dropout in baseline
        )
        
        # Removed: Dropout layer (baseline has no regularization)
        # self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Get dimensions
        batch_size = embedded.size(0)
        seq_len = embedded.size(1)
        
        # Handle text_lengths: ensure it's a 1D tensor with batch_size elements
        text_lengths_flat = text_lengths.flatten().cpu().long()
        
        # text_lengths should have exactly batch_size elements (one length per batch item)
        if len(text_lengths_flat) != batch_size:
            raise ValueError(
                f"text_lengths size mismatch: got {len(text_lengths_flat)} elements, "
                f"expected {batch_size} (batch_size). text_lengths.shape={text_lengths.shape}, "
                f"text.shape={text.shape}, embedded.shape={embedded.shape}"
            )
        
        # Clamp lengths to be at most the sequence length (safety check)
        text_lengths_clamped = torch.clamp(text_lengths_flat, min=1, max=seq_len)
        
        # Pack the padded sequences for efficient processing
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths_clamped, batch_first=True, enforce_sorted=False
        )
        
        # Pass through RNN
        packed_output, hidden = self.rnn(packed_embedded)
        # hidden: [n_layers, batch_size, hidden_dim]
        
        # Use the last hidden state from the last layer
        last_hidden = hidden[-1]  # [batch_size, hidden_dim]
        
        # Removed: Apply dropout (baseline has no regularization)
        # last_hidden = self.dropout(last_hidden)
        
        # Pass through fully connected layer
        output = self.fc(last_hidden)  # [batch_size, output_dim]
        
        return output


# Utility function for counting parameters
def count_parameters(model):
    """Count trainable parameters"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Training and evaluation functions removed - code is now inline below for easier debugging

In [ ]:
print(len(TEXT.vocab))

Training order:
1. Word aggregation
2. Hyperparameters tuning
3. Regularization

In [ ]:
# ============================================================================
# Part 2: Initial Simple RNN Model Training
# ============================================================================

print("\n" + "="*80)
print("PART 2: SIMPLE RNN MODEL TRAINING")
print("="*80)

# Get pretrained embeddings from Part 1 (frozen embeddings)
pretrained_embeddings = fasttext.weight.data

# Get embedding dimension and vocab size from the fasttext embedding layer
embedding_dim = fasttext.weight.shape[1]
fasttext_vocab_size = fasttext.weight.shape[0]  # Vocab size from saved embedding

# Verify vocab sizes match (they might differ if vocab was rebuilt)
print(f"TEXT.vocab size: {len(TEXT.vocab)}")
print(f"FastText embedding vocab size: {fasttext_vocab_size}")

# Hyperparameters
BATCH_SIZE = 64
HIDDEN_DIM = 256
N_LAYERS = 1
DROPOUT = 0.0  # Baseline: no dropout
N_EPOCHS = 100
LEARNING_RATE = 0.001
# Removed: PATIENCE = 10  # Baseline: no early stopping

# Create data iterators (inline for easier debugging)
# Note: Different sequence lengths per batch are normal - BucketIterator groups similar-length sequences
train_iterator = data.BucketIterator(
    train_data,
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,  # Shuffle for training
    device=device
)

val_iterator = data.BucketIterator(
    validation_data,
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,  # No shuffle for validation (deterministic)
    device=device
)

test_iterator = data.BucketIterator(
    test_data,
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,  # No shuffle for test (deterministic)
    device=device
)

# Initialize simple RNN model (Baseline)
# Use vocab size from loaded embedding to match the saved weights exactly
model = SimpleRNNClassifier(
    vocab_size=fasttext_vocab_size,  # Must match saved embedding vocab size
    embedding_dim=embedding_dim,
    hidden_dim=HIDDEN_DIM,
    output_dim=num_classes,
    n_layers=N_LAYERS,
    dropout=0.0,  # Baseline: no dropout
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\n>>> Training Baseline RNN Model")
print(f"Configuration:")
print(f"  - Hidden Dim: {HIDDEN_DIM}")
print(f"  - Layers: {N_LAYERS}")
print(f"  - Dropout: {DROPOUT} (Baseline: no regularization)")
print(f"  - Learning Rate: {LEARNING_RATE}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Epochs: {N_EPOCHS} (no early stopping)")
print(f"  - Embedding Dim: {embedding_dim} (FastText)")
print(f"  - Embeddings: LEARNABLE (updated during training)")
print(f"  - OOV Handling: FastText subword embeddings + trainable <unk> token")

# ============================================================================
# Helper function to process batches consistently
# ============================================================================

def process_batch(batch, debug=False):
    """
    Process a batch from BucketIterator, handling text transpose correctly.
    Returns: text, text_lengths, labels (all properly formatted)
    """
    text, text_lengths = batch.text
    labels = batch.label
    
    if debug:
        print(f"DEBUG BATCH - text shape: {text.shape}, text_lengths shape: {text_lengths.shape}, labels shape: {labels.shape}")
    
    # torchtext BucketIterator returns text as [seq_len, batch_size] by default
    # We need [batch_size, seq_len] for batch_first=True in the model
    expected_batch_size = labels.shape[0]
    
    if text.dim() == 2:
        if text.shape[1] == expected_batch_size and len(text_lengths) == expected_batch_size:
            # text is [seq_len, batch_size], transpose to [batch_size, seq_len]
            text = text.transpose(0, 1)
            if debug:
                print(f"DEBUG BATCH - Transposed text to [batch_size, seq_len]: {text.shape}")
        elif text.shape[0] == expected_batch_size and len(text_lengths) == expected_batch_size:
            # text is already [batch_size, seq_len]
            if debug:
                print(f"DEBUG BATCH - text already in correct format: {text.shape}")
        else:
            raise ValueError(
                f"Cannot determine text format: text.shape={text.shape}, "
                f"text_lengths.shape={text_lengths.shape}, labels.shape={labels.shape}"
            )
    
    # Verify dimensions match
    assert text.shape[0] == len(text_lengths) == labels.shape[0], \
        f"Batch size mismatch: text.shape[0]={text.shape[0]}, len(text_lengths)={len(text_lengths)}, labels.shape[0]={labels.shape[0]}"
    
    return text, text_lengths, labels

# Set seeds for reproducibility
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Note: PyTorch/IPython compatibility fix is already applied in cell 0
# The torch._dynamo.disable decorator has been patched to handle the 'wrapping' parameter

# ============================================================================
# Training Loop (inline for easier debugging)
# ============================================================================

best_val_acc = 0
# Removed: patience_counter = 0  # Baseline: no early stopping

train_losses = []
train_accs = []
val_losses = []
val_accs = []

print(f"\nStarting training for {N_EPOCHS} epochs...")  # Removed "up to" - no early stopping
print(f"Device: {device}")
print(f"Trainable parameters: {count_parameters(model):,}")
print(f"Embedding layer learnable: {model.embedding.weight.requires_grad}")
# Removed: print(f"Early stopping patience: {PATIENCE} epochs")  # Baseline: no early stopping
print("-" * 80)

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    # ========================================================================
    # Training for one epoch (inline)
    # ========================================================================
    model.train()
    train_epoch_loss = 0
    train_all_preds = []
    train_all_labels = []
    
    batch_idx = 0
    for batch in train_iterator:
        # Process batch (with debug only for first batch)
        text, text_lengths, labels = process_batch(batch, debug=(batch_idx == 0))
        batch_idx += 1
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(text, text_lengths)
        
        # Calculate loss
        loss = criterion(predictions, labels)
        
        # Backward pass
        loss.backward()
        
        # Removed: Gradient clipping (baseline has no gradient clipping)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update weights
        optimizer.step()
        
        train_epoch_loss += loss.item()
        
        # Store predictions and labels for metrics
        preds = torch.argmax(predictions, dim=1)
        train_all_preds.extend(preds.cpu().numpy())
        train_all_labels.extend(labels.cpu().numpy())
    
    # Calculate training accuracy
    train_acc = accuracy_score(train_all_labels, train_all_preds)
    train_loss = train_epoch_loss / len(train_iterator)
    
    # ========================================================================
    # Validation evaluation (inline)
    # ========================================================================
    model.eval()
    val_epoch_loss = 0
    val_all_preds = []
    val_all_labels = []
    
    with torch.no_grad():
        for batch in val_iterator:
            # Process batch consistently with training
            text, text_lengths, labels = process_batch(batch, debug=False)
            
            # Forward pass
            predictions = model(text, text_lengths)
            
            # Calculate loss
            loss = criterion(predictions, labels)
            val_epoch_loss += loss.item()
            
            # Store predictions and labels
            preds = torch.argmax(predictions, dim=1)
            val_all_preds.extend(preds.cpu().numpy())
            val_all_labels.extend(labels.cpu().numpy())
    
    # Calculate validation accuracy
    val_acc = accuracy_score(val_all_labels, val_all_preds)
    val_loss = val_epoch_loss / len(val_iterator)
    
    # ========================================================================
    # Logging (without early stopping)
    # ========================================================================
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    # Store metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f'Epoch: {epoch+1:02}/{N_EPOCHS} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%')
    
    # Track best model (but don't stop early - baseline trains for all epochs)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # Save best model
        torch.save(model.state_dict(), 'rnn_simple_best.pt')
        print(f'\t>>> New best model saved with Val Acc: {val_acc*100:.2f}%')
    # Removed: Early stopping break logic (baseline trains for all epochs)

print("-" * 80)
print(f"Training completed! Best validation accuracy: {best_val_acc*100:.2f}%")
print(f"Total epochs trained: {N_EPOCHS}")

In [ ]:
# ============================================================================
# Validation Set Evaluation (inline) - Evaluate best model on validation set
# ============================================================================

print("\n" + "="*80)
print("VALIDATION SET EVALUATION (Best Model)")
print("="*80)

# Load best model and evaluate on validation set
model.load_state_dict(torch.load('rnn_simple_best.pt'))

model.eval()
val_eval_loss = 0
val_eval_preds = []
val_eval_labels = []
val_eval_probs = []  # Store probabilities for AUC-ROC

with torch.no_grad():
    for batch in val_iterator:
        text, text_lengths = batch.text
        labels = batch.label
        
        # Process batch consistently
        text, text_lengths, labels = process_batch(batch, debug=False)
        
        # Forward pass
        predictions = model(text, text_lengths)
        
        # Calculate loss
        loss = criterion(predictions, labels)
        val_eval_loss += loss.item()
        
        # Store predictions, labels, and probabilities
        probs = torch.softmax(predictions, dim=1)
        preds = torch.argmax(predictions, dim=1)
        val_eval_preds.extend(preds.cpu().numpy())
        val_eval_labels.extend(labels.cpu().numpy())
        val_eval_probs.extend(probs.cpu().numpy())

# Calculate validation metrics
val_eval_acc = accuracy_score(val_eval_labels, val_eval_preds)
val_eval_f1 = f1_score(val_eval_labels, val_eval_preds, average='weighted')
val_eval_loss_final = val_eval_loss / len(val_iterator)

# Calculate AUC-ROC (one-vs-rest for multiclass)
try:
    val_eval_probs_array = np.array(val_eval_probs)
    val_eval_labels_bin = label_binarize(val_eval_labels, classes=range(num_classes))
    val_eval_auc = roc_auc_score(val_eval_labels_bin, val_eval_probs_array, average='weighted', multi_class='ovr')
except Exception as e:
    print(f"Warning: Could not calculate AUC-ROC: {e}")
    val_eval_auc = 0.0

print(f"\n>>> Validation Set Results (Best Model):")
print(f"Validation Loss: {val_eval_loss_final:.4f}")
print(f"Validation Accuracy: {val_eval_acc*100:.2f}%")
print(f"Validation F1 Score: {val_eval_f1:.4f}")
print(f"Validation AUC-ROC: {val_eval_auc:.4f}")

# ============================================================================
# Test Set Evaluation (inline)
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION")
print("="*80)

model.eval()
test_epoch_loss = 0
test_all_preds = []
test_all_labels = []
test_all_probs = []  # Store probabilities for AUC-ROC

with torch.no_grad():
    for batch in test_iterator:
        text, text_lengths = batch.text
        labels = batch.label
        
        # Process batch consistently
        text, text_lengths, labels = process_batch(batch, debug=False)
        
        # Forward pass
        predictions = model(text, text_lengths)
        
        # Calculate loss
        loss = criterion(predictions, labels)
        test_epoch_loss += loss.item()
        
        # Store predictions, labels, and probabilities
        probs = torch.softmax(predictions, dim=1)
        preds = torch.argmax(predictions, dim=1)
        test_all_preds.extend(preds.cpu().numpy())
        test_all_labels.extend(labels.cpu().numpy())
        test_all_probs.extend(probs.cpu().numpy())

# Calculate test metrics
test_acc = accuracy_score(test_all_labels, test_all_preds)
test_f1 = f1_score(test_all_labels, test_all_preds, average='weighted')
test_loss = test_epoch_loss / len(test_iterator)

# Calculate AUC-ROC (one-vs-rest for multiclass)
try:
    test_all_probs_array = np.array(test_all_probs)
    test_all_labels_bin = label_binarize(test_all_labels, classes=range(num_classes))
    test_auc = roc_auc_score(test_all_labels_bin, test_all_probs_array, average='weighted', multi_class='ovr')
except Exception as e:
    print(f"Warning: Could not calculate AUC-ROC: {e}")
    test_auc = 0.0

print(f"\n>>> Test Set Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test AUC-ROC: {test_auc:.4f}")

print("\n" + "="*80)
print("PART 2 INITIAL TRAINING COMPLETE")
print("="*80)

In [ ]:
# ============================================================================
# Part 2.2: Sequential Hyperparameter Tuning (One Variable at a Time)
# ============================================================================

print("\n" + "="*80)
print("PART 2.2: SEQUENTIAL HYPERPARAMETER TUNING")
print("="*80)

# ============================================================================
# ============================================================================
# Step 0: Epoch + Early Stopping Configuration Tuning
# ============================================================================
print("\n" + "="*80)
print("STEP 0: EPOCH + EARLY STOPPING TUNING")
print("="*80)
print("Testing different MAX_EPOCHS and PATIENCE configurations")

# Test different epoch and patience configurations
max_epochs_options = [100, 200, 300]
patience = 10

# Use baseline config for testing epoch settings
baseline_config = {
    'lr': 0.001,
    'batch_size': 64,
    'hidden_dim': 256,
    'optimizer': 'Adam'
}

step0_configs = []
for max_epochs in max_epochs_options:
        step0_configs.append({
            'config': baseline_config.copy(),
            'max_epochs': max_epochs,
            'patience': patience
        })

print(f"Total combinations to test: {len(step0_configs)}")
print("Combinations (Max_Epochs, Patience):")
for idx, ep_config in enumerate(step0_configs, 1):
    print(f"  {idx}. Max_Epochs={ep_config['max_epochs']}, Patience={ep_config['patience']}")

# Helper function to train with specific epoch/patience settings
def train_and_evaluate_with_epochs(config, max_epochs, patience, config_name="config"):
    """Train a model for specific number of epochs WITHOUT early stopping"""
    print(f"\n>>> Testing: {config_name}")
    print(f"    LR={config['lr']}, Batch={config['batch_size']}, Hidden={config['hidden_dim']}, Opt={config['optimizer']}")
    print(f"    Epochs={max_epochs} (NO early stopping - training for full {max_epochs} epochs)")
    
    # Create iterators with specific batch size
    train_iter = data.BucketIterator(
        train_data,
        batch_size=config['batch_size'],
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=True,
        device=device
    )
    
    val_iter = data.BucketIterator(
        validation_data,
        batch_size=config['batch_size'],









        
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=False,
        device=device
    )
    
    # Reset random seeds INSIDE function to ensure fresh model for each config
    # This is critical to ensure each max_epochs config starts from scratch
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
    # Ensure deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Create model with specific hidden dimension
    model = SimpleRNNClassifier(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=config['hidden_dim'],
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=0.0,  # Baseline: no dropout
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings
    ).to(device)
    
    # Select optimizer
    if config['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)
    elif config['optimizer'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=config['lr'])
    
    # Training loop WITHOUT early stopping - train for full num_epochs
    best_val_acc = 0.0
    best_val_acc_at_epoch = 0
    final_val_acc = 0.0
    
    for epoch in range(max_epochs):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        # Track best validation accuracy (but don't stop early)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_acc_at_epoch = epoch + 1
        
        final_val_acc = val_acc  # Store final epoch's validation accuracy
        
        # Optional: print progress every 10 epochs or at the end
        if (epoch + 1) % 10 == 0 or (epoch + 1) == max_epochs:
            print(f"    Epoch {epoch+1}/{max_epochs}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    print(f"    Final Val Acc: {final_val_acc*100:.2f}% | Best Val Acc: {best_val_acc*100:.2f}% (at epoch {best_val_acc_at_epoch}/{max_epochs})")
    return best_val_acc, best_val_acc_at_epoch, max_epochs

step0_results = []
for idx, ep_config in enumerate(step0_configs):
    # Set fixed seed for reproducibility - ensures consistent batch ordering
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
    
    val_acc, best_epoch, total_epochs = train_and_evaluate_with_epochs(
        ep_config['config'],
        ep_config['max_epochs'],
        ep_config['patience'],
        f"Step 0 Config {idx+1}/{len(step0_configs)}"
    )
    step0_results.append({
        'num_epochs': ep_config['max_epochs'],
        'val_acc': val_acc,
        'best_epoch': best_epoch,
        'total_epochs': total_epochs
    })

# Find best epoch configuration
best_step0 = max(step0_results, key=lambda x: x['val_acc'])
BEST_EPOCHS = best_step0['num_epochs']

# Set appropriate MAX_EPOCHS and PATIENCE for subsequent steps
# Use the best number of epochs with some buffer, and set a reasonable patience
MAX_EPOCHS = BEST_EPOCHS
PATIENCE = 7  # Default patience for early stopping in subsequent steps

print(f"\n>>> Step 0 Results:")
print(f"{'#':<4} {'Epochs':<8} {'Val Acc':<10} {'Best At Epoch':<15} {'Total Trained':<15}")
print("-" * 60)
for idx, result in enumerate(step0_results, 1):
    print(f"{idx:<4} {result['num_epochs']:<8} {result['val_acc']*100:<10.2f}% {result['best_epoch']:<15} {result['total_epochs']:<15}")
print(f"\n>>> Best from Step 0: Epochs={BEST_EPOCHS}, Val Acc={best_step0['val_acc']*100:.2f}%")
print(f"    Best validation accuracy was achieved at epoch {best_step0['best_epoch']} out of {best_step0['total_epochs']}")
print(f"\n>>> Using MAX_EPOCHS={MAX_EPOCHS} and PATIENCE={PATIENCE} for subsequent steps (with early stopping)")

# Helper function to train and evaluate a model configuration
def train_and_evaluate(config, config_name="config"):
    """Train a model with given configuration and return validation accuracy"""
    print(f"\n>>> Testing: {config_name}")
    print(f"    LR={config['lr']}, Batch={config['batch_size']}, Hidden={config['hidden_dim']}, Opt={config['optimizer']}")
    
    # Create iterators with specific batch size
    train_iter = data.BucketIterator(
        train_data,
        batch_size=config['batch_size'],
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=True,
        device=device
    )
    
    val_iter = data.BucketIterator(
        validation_data,
        batch_size=config['batch_size'],
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=False,
        device=device
    )
    
    # Create model with specific hidden dimension
    model = SimpleRNNClassifier(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=config['hidden_dim'],
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=0.0,  # Baseline: no dropout
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings
    ).to(device)
    
    # Select optimizer
    if config['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=config['lr'])
    
    # Training loop with early stopping
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
    
    print(f"    Best Val Acc: {best_val_acc*100:.2f}% (stopped at epoch {epoch+1})")
    return best_val_acc, epoch + 1






In [ ]:
# ============================================================================
# Step 1: Group 1 - Learning Rate + Batch Size (Test Together)
# ============================================================================

# Helper function to train and evaluate a model configuration (uses best MAX_EPOCHS and PATIENCE from Step 0)
def train_and_evaluate(config, config_name="config"):
    """Train a model with given configuration and return validation accuracy"""
    print(f"\n>>> Testing: {config_name}")
    print(f"    LR={config['lr']}, Batch={config['batch_size']}, Hidden={config['hidden_dim']}, Opt={config['optimizer']}")
    
    # Create iterators with specific batch size
    train_iter = data.BucketIterator(
        train_data,
        batch_size=config['batch_size'],
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=True,
        device=device
    )
    
    val_iter = data.BucketIterator(
        validation_data,
        batch_size=config['batch_size'],
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=False,
        device=device
    )
    
    # Create model with specific hidden dimension
    model = SimpleRNNClassifier(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=config['hidden_dim'],
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=0.0,  # Baseline: no dropout
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings
    ).to(device)
    
    # Select optimizer
    if config['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)
    elif config['optimizer'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=config['lr'])
    
    # Training loop with early stopping (using best MAX_EPOCHS and PATIENCE from Step 0)
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
    
    print(f"    Best Val Acc: {best_val_acc*100:.2f}% (stopped at epoch {epoch+1})")
    return best_val_acc, epoch + 1
    
print("\n" + "="*80)
print("STEP 1: LEARNING RATE + BATCH SIZE TUNING")
print("="*80)
print("Testing ALL combinations of LR and Batch Size (they interact)")

# Test all combinations of learning rates and batch sizes
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

step1_configs = []
for lr in learning_rates:
    for batch_size in batch_sizes:
        step1_configs.append({
            'lr': lr,
            'batch_size': batch_size,
            'hidden_dim': 256,
            'optimizer': 'Adam'
        })

print(f"Total combinations to test: {len(step1_configs)}")
print("Combinations:")
for idx, config in enumerate(step1_configs, 1):
    print(f"  {idx}. LR={config['lr']}, Batch={config['batch_size']}")

step1_results = []
for idx, config in enumerate(step1_configs):
    # Set fixed seed for reproducibility
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
    
    val_acc, epoch_stopped = train_and_evaluate(config, f"Step 1 Config {idx+1}/{len(step1_configs)}")
    step1_results.append({
        'config': config,
        'val_acc': val_acc,
        'epoch_stopped': epoch_stopped
    })

# Find best LR + Batch Size
best_step1 = max(step1_results, key=lambda x: x['val_acc'])
best_lr = best_step1['config']['lr']
best_batch_size = best_step1['config']['batch_size']

print(f"\n>>> Step 1 Results:")
print(f"{'#':<4} {'LR':<8} {'Batch':<7} {'Val Acc':<10} {'Epochs':<7}")
print("-" * 40)
for idx, result in enumerate(step1_results):
    c = result['config']
    print(f"{idx+1:<4} {c['lr']:<8} {c['batch_size']:<7} {result['val_acc']*100:<10.2f}% {result['epoch_stopped']:<7}")
print(f"\n>>> Best from Step 1: LR={best_lr}, Batch={best_batch_size}, Val Acc={best_step1['val_acc']*100:.2f}%")

In [ ]:
# ============================================================================
# Step 2: Group 2 - Optimizer
# ============================================================================
print("\n" + "="*80)
print("STEP 2: OPTIMIZER")
print("="*80)
print(f"Using best LR={best_lr} and Batch={best_batch_size} from Step 1")

step2_configs = [
    {'lr': best_lr, 'batch_size': best_batch_size, 'hidden_dim': 256, 'optimizer': 'Adam'},
    {'lr': best_lr, 'batch_size': best_batch_size, 'hidden_dim': 256, 'optimizer': 'SGD'},
    {'lr': best_lr, 'batch_size': best_batch_size, 'hidden_dim': 256, 'optimizer': 'RMSprop'},
    {'lr': best_lr, 'batch_size': best_batch_size, 'hidden_dim': 256, 'optimizer': 'Adagrad'},
]

step2_results = []
for idx, config in enumerate(step2_configs):
    # Set fixed seed for reproducibility
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
    
    val_acc, epoch_stopped = train_and_evaluate(config, f"Step 2 Config {idx+1}/{len(step2_configs)}")
    step2_results.append({
        'config': config,
        'val_acc': val_acc,
        'epoch_stopped': epoch_stopped
    })

# Find best Optimizer (and potentially adjusted LR)
best_step2 = max(step2_results, key=lambda x: x['val_acc'])
best_optimizer = best_step2['config']['optimizer']
final_lr = best_step2['config']['lr']  # May be different if SGD needed higher LR

print(f"\n>>> Step 2 Results:")
print(f"{'#':<4} {'LR':<8} {'Optimizer':<10} {'Val Acc':<10} {'Epochs':<7}")
print("-" * 45)
for idx, result in enumerate(step2_results):
    c = result['config']
    print(f"{idx+1:<4} {c['lr']:<8} {c['optimizer']:<10} {result['val_acc']*100:<10.2f}% {result['epoch_stopped']:<7}")
print(f"\n>>> Best from Step 2: LR={final_lr}, Optimizer={best_optimizer}, Val Acc={best_step2['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Step 3: Hidden Dimension (Test Independently)
# ============================================================================
print("\n" + "="*80)
print("STEP 3: HIDDEN DIMENSION TUNING")
print("="*80)
print(f"Using best LR={final_lr}, Batch={best_batch_size}, Optimizer={best_optimizer} from Steps 1-2")

step3_configs = [
    {'lr': final_lr, 'batch_size': best_batch_size, 'hidden_dim': 128, 'optimizer': best_optimizer},
    {'lr': final_lr, 'batch_size': best_batch_size, 'hidden_dim': 256, 'optimizer': best_optimizer},
    {'lr': final_lr, 'batch_size': best_batch_size, 'hidden_dim': 512, 'optimizer': best_optimizer},
]

step3_results = []
for idx, config in enumerate(step3_configs):
    # Set fixed seed for reproducibility
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
    
    val_acc, epoch_stopped = train_and_evaluate(config, f"Step 3 Config {idx+1}/{len(step3_configs)}")
    step3_results.append({
        'config': config,
        'val_acc': val_acc,
        'epoch_stopped': epoch_stopped
    })

# Find best Hidden Dimension
best_step3 = max(step3_results, key=lambda x: x['val_acc'])
best_hidden_dim = best_step3['config']['hidden_dim']

print(f"\n>>> Step 3 Results:")
print(f"{'#':<4} {'Hidden Dim':<12} {'Val Acc':<10} {'Epochs':<7}")
print("-" * 35)
for idx, result in enumerate(step3_results):
    c = result['config']
    print(f"{idx+1:<4} {c['hidden_dim']:<12} {result['val_acc']*100:<10.2f}% {result['epoch_stopped']:<7}")
print(f"\n>>> Best from Step 3: Hidden Dim={best_hidden_dim}, Val Acc={best_step3['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Final Best Configuration Summary
# ============================================================================
print("\n" + "="*80)
print("HYPERPARAMETER TUNING COMPLETE - FINAL BEST CONFIGURATION")
print("="*80)

final_best_config = {
    'lr': final_lr,
    'batch_size': best_batch_size,
    'hidden_dim': best_hidden_dim,
    'optimizer': best_optimizer,
    'max_epochs': MAX_EPOCHS,
    'patience': PATIENCE
}

print(f"\n>>> Best Configuration Found:")
print(f"    Learning Rate: {final_best_config['lr']}")
print(f"    Batch Size: {final_best_config['batch_size']}")
print(f"    Hidden Dimension: {final_best_config['hidden_dim']}")
print(f"    Optimizer: {final_best_config['optimizer']}")
print(f"    Max Epochs: {final_best_config['max_epochs']} (with early stopping, patience={final_best_config['patience']})")
print(f"    Best Validation Accuracy: {best_step3['val_acc']*100:.2f}%")

print("\n" + "="*80)
print("SEQUENTIAL HYPERPARAMETER TUNING COMPLETE")
print("="*80)


In [ ]:
# ============================================================================
# Word Aggregation Method Comparison
# ============================================================================
print("\n" + "="*80)
print("WORD AGGREGATION METHOD COMPARISON")
print("="*80)
print(f"Using best hyperparameters from tuning:")
print(f"    LR={final_lr}, Batch={best_batch_size}, Hidden={best_hidden_dim}, Optimizer={best_optimizer}")
print(f"    Max Epochs={MAX_EPOCHS}, Patience={PATIENCE}")

# Extended RNN Classifier with multiple aggregation methods
class RNN_Classifier_Aggregation(nn.Module):
    """
    RNN for topic classification with multiple aggregation strategies.
    Uses pretrained embeddings (learnable/updated during training).
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=1, dropout=0.0, padding_idx=0, pretrained_embeddings=None,
                 aggregation='last'):
        super(RNN_Classifier_Aggregation, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.aggregation = aggregation  # 'last', 'mean', 'max', 'attention'
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # RNN layer
        self.rnn = nn.RNN(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.0  # No dropout in baseline
        )
        
        # Attention mechanism for aggregation (only created if needed)
        if aggregation == 'attention':
            self.attention = nn.Linear(hidden_dim, 1)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Get dimensions
        batch_size = embedded.size(0)
        seq_len = embedded.size(1)
        
        # Handle text_lengths
        text_lengths_flat = text_lengths.flatten().cpu().long()
        if len(text_lengths_flat) != batch_size:
            raise ValueError(
                f"text_lengths size mismatch: got {len(text_lengths_flat)} elements, "
                f"expected {batch_size}"
            )
        
        # Clamp lengths
        text_lengths_clamped = torch.clamp(text_lengths_flat, min=1, max=seq_len)
        
        # Move to same device as text for mask operations later
        # Keep CPU version for pack_padded_sequence (requires CPU)
        # Create device version for mask operations later
        text_lengths_clamped_device = text_lengths_clamped.to(text.device)
        # Pack the padded sequences
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths_clamped, batch_first=True, enforce_sorted=False
        )
        
        # Pass through RNN
        packed_output, hidden = self.rnn(packed_embedded)
        # hidden: [n_layers, batch_size, hidden_dim]
        
        # Aggregate word representations to sentence representation
        if self.aggregation == 'last':
            # Use the last hidden state from the last layer
            sentence_repr = hidden[-1]  # [batch_size, hidden_dim]
            
        elif self.aggregation == 'mean':
            # Mean pooling over all outputs (ignoring padding)
            # Unpack the sequences first
            output, output_lengths = nn.utils.rnn.pad_packed_sequence(
                packed_output, batch_first=True
            )
            # output: [batch_size, seq_len, hidden_dim]
            
            # Create mask for padding
            mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths_clamped_device.unsqueeze(1)
            mask = mask.unsqueeze(2).float()  # [batch_size, seq_len, 1]
            
            # Apply mask and compute mean
            masked_output = output * mask
            sum_output = masked_output.sum(dim=1)  # [batch_size, hidden_dim]
            sentence_repr = sum_output / text_lengths_clamped_device.unsqueeze(1).float()
            
        elif self.aggregation == 'max':
            # Max pooling over all outputs
            output, output_lengths = nn.utils.rnn.pad_packed_sequence(
                packed_output, batch_first=True
            )
            # output: [batch_size, seq_len, hidden_dim]
            
            # Create mask for padding (set padding to -inf before max)
            mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths_clamped_device.unsqueeze(1)
            mask = mask.unsqueeze(2).float()  # [batch_size, seq_len, 1]
            
            masked_output = output * mask + (1 - mask) * float('-inf')
            sentence_repr, _ = torch.max(masked_output, dim=1)
            
        elif self.aggregation == 'attention':
            # Attention mechanism
            output, output_lengths = nn.utils.rnn.pad_packed_sequence(
                packed_output, batch_first=True
            )
            # output: [batch_size, seq_len, hidden_dim]
            
            # Compute attention scores
            attn_scores = self.attention(output).squeeze(2)  # [batch_size, seq_len]
            
            # Mask padding positions
            mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths_clamped_device.unsqueeze(1)
            attn_scores = attn_scores.masked_fill(~mask, float('-inf'))
            
            # Apply softmax
            attn_weights = torch.softmax(attn_scores, dim=1).unsqueeze(1)  # [batch_size, 1, seq_len]
            
            # Weighted sum
            sentence_repr = torch.bmm(attn_weights, output).squeeze(1)  # [batch_size, hidden_dim]
        
        # Pass through fully connected layer
        output = self.fc(sentence_repr)  # [batch_size, output_dim]
        
        return output

# Test different aggregation methods
aggregation_methods = ['last', 'mean', 'max', 'attention']

print(f"\nTesting {len(aggregation_methods)} aggregation methods:")
for method in aggregation_methods:
    print(f"  - {method}")

aggregation_results = []

for agg_method in aggregation_methods:
    print(f"\n{'='*80}")
    print(f"Testing Aggregation Method: {agg_method.upper()}")
    print(f"{'='*80}")
    
    # Set fixed seed for reproducibility - ensures consistent batch ordering
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
    
    # Create iterators with best batch size
    train_iter = data.BucketIterator(
        train_data,
        batch_size=best_batch_size,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=True,
        device=device
    )
    
    val_iter = data.BucketIterator(
        validation_data,
        batch_size=best_batch_size,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=False,
        device=device
    )
    
    test_iter = data.BucketIterator(
        test_data,
        batch_size=best_batch_size,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=False,
        device=device
    )
    
    # Create model with best hyperparameters and specific aggregation method
    model = RNN_Classifier_Aggregation(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=best_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=0.0,  # Baseline: no dropout
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=agg_method
    ).to(device)
    
    # Select optimizer with best learning rate
    if best_optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=final_lr)
    elif best_optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9)
    elif best_optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=final_lr)
    elif best_optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=final_lr)
    
    # Training loop with early stopping
    best_val_acc = 0.0
    patience_counter = 0
    
    print(f"\n>>> Training model with {agg_method} aggregation...")
    
    for epoch in range(MAX_EPOCHS):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model for this aggregation method
            torch.save(model.state_dict(), f'rnn_agg_{agg_method}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
        
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    # Load best model and evaluate on test set
    model.load_state_dict(torch.load(f'rnn_agg_{agg_method}_best.pt'))
    model.eval()
    
    test_loss = 0
    test_preds = []
    test_labels = []
    test_probs = []
    
    with torch.no_grad():
        for batch in test_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
            
            probs = torch.softmax(predictions, dim=1)
            preds = torch.argmax(predictions, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
            test_probs.extend(probs.cpu().numpy())
    
    # Calculate test metrics
    test_acc = accuracy_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds, average='weighted')
    test_loss_avg = test_loss / len(test_iter)
    
    # Calculate AUC-ROC
    try:
        test_probs_array = np.array(test_probs)
        test_labels_bin = label_binarize(test_labels, classes=range(num_classes))
        test_auc = roc_auc_score(test_labels_bin, test_probs_array, average='weighted', multi_class='ovr')
    except Exception as e:
        print(f"    Warning: Could not calculate AUC-ROC: {e}")
        test_auc = 0.0
    
    aggregation_results.append({
        'method': agg_method,
        'val_acc': best_val_acc,
        'test_acc': test_acc,
        'test_f1': test_f1,
        'test_auc': test_auc,
        'test_loss': test_loss_avg
    })
    
    print(f"\n>>> Results for {agg_method} aggregation:")
    print(f"    Validation Acc: {best_val_acc*100:.2f}%")
    print(f"    Test Acc: {test_acc*100:.2f}%")
    print(f"    Test F1: {test_f1:.4f}")
    print(f"    Test AUC-ROC: {test_auc:.4f}")

# Print summary table
print(f"\n{'='*80}")
print("AGGREGATION METHOD COMPARISON - RESULTS SUMMARY")
print(f"{'='*80}")

print(f"\n>>> Results Summary:")
print(f"{'Method':<12} {'Val Acc':<10} {'Test Acc':<10} {'Test F1':<10} {'Test AUC':<10}")
print("-" * 55)
for result in aggregation_results:
    print(f"{result['method']:<12} {result['val_acc']*100:<10.2f}% {result['test_acc']*100:<10.2f}% "
          f"{result['test_f1']:<10.4f} {result['test_auc']:<10.4f}")

# Find best aggregation method
best_aggregation = max(aggregation_results, key=lambda x: x['val_acc'])

print(f"\n>>> Best Aggregation Method: {best_aggregation['method'].upper()}")
print(f"    Validation Accuracy: {best_aggregation['val_acc']*100:.2f}%")
print(f"    Test Accuracy: {best_aggregation['test_acc']*100:.2f}%")
print(f"    Test F1 Score: {best_aggregation['test_f1']:.4f}")
print(f"    Test AUC-ROC: {best_aggregation['test_auc']:.4f}")

print(f"\n{'='*80}")
print("AGGREGATION METHOD COMPARISON COMPLETE")
print(f"{'='*80}")


In [ ]:
# ============================================================================
# Regularization: Baseline (No Regularization)
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION STEP 0: BASELINE (NO REGULARIZATION)")
print("="*80)
print(f"Using best hyperparameters from tuning:")
print(f"    LR={final_lr}, Batch={best_batch_size}, Hidden={best_hidden_dim}, Optimizer={best_optimizer}")
print(f"    Max Epochs={MAX_EPOCHS}, Patience={PATIENCE}")
print(f"    Best Aggregation Method: {best_aggregation['method'].upper()}")

# Baseline configuration
baseline_config = {
    'dropout': 0.0,
    'grad_clip': 0.0,
    'l1_lambda': 0.0,
    'l2_lambda': 0.0
}

print(f"\nBaseline Configuration:")
print(f"    Dropout: {baseline_config['dropout']}")
print(f"    Gradient Clipping: {baseline_config['grad_clip']}")
print(f"    L1 Lambda: {baseline_config['l1_lambda']}")
print(f"    L2 Lambda: {baseline_config['l2_lambda']}")

# Create iterators
train_iter = data.BucketIterator(
    train_data,
    batch_size=best_batch_size,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter = data.BucketIterator(
    validation_data,
    batch_size=best_batch_size,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter = data.BucketIterator(
    test_data,
    batch_size=best_batch_size,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create model
model = RNN_Classifier_Aggregation(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=best_hidden_dim,
    output_dim=num_classes,
    n_layers=N_LAYERS,
    dropout=baseline_config['dropout'],
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings,
    aggregation=best_aggregation['method']
).to(device)

# Select optimizer
if best_optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=baseline_config['l2_lambda'])
elif best_optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=baseline_config['l2_lambda'])
elif best_optimizer == 'RMSprop':
    optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=baseline_config['l2_lambda'])
elif best_optimizer == 'Adagrad':
    optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=baseline_config['l2_lambda'])

# Helper function for L1 regularization
def compute_l1_loss(model, l1_lambda):
    """Compute L1 regularization loss"""
    if l1_lambda > 0:
        return l1_lambda * sum(p.abs().sum() for p in model.parameters() if p.requires_grad)
    return 0.0

# Training loop
best_val_acc = 0.0
patience_counter = 0

print(f"\n>>> Training baseline model...")

for epoch in range(MAX_EPOCHS):
    # Training
    model.train()
    train_loss = 0
    train_preds = []
    train_labels = []
    
    for batch in train_iter:
        text, text_lengths, labels = process_batch(batch, debug=False)
        optimizer.zero_grad()
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions, labels)
        
        # Add L1 regularization
        if baseline_config['l1_lambda'] > 0:
            loss = loss + compute_l1_loss(model, baseline_config['l1_lambda'])
        
        loss.backward()
        
        # Gradient clipping
        if baseline_config['grad_clip'] > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), baseline_config['grad_clip'])
        
        optimizer.step()
        
        train_loss += loss.item()
        preds = torch.argmax(predictions, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
    
    train_acc = accuracy_score(train_labels, train_preds)
    
    # Validation
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for batch in val_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            val_loss += loss.item()
            
            preds = torch.argmax(predictions, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_acc = accuracy_score(val_labels, val_preds)
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'rnn_reg_baseline_best.pt')
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
            break
    
    if (epoch + 1) % 10 == 0:
        print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")

# Evaluate on test set
model.load_state_dict(torch.load('rnn_reg_baseline_best.pt'))
model.eval()

test_loss = 0
test_preds = []
test_labels = []
test_probs = []

with torch.no_grad():
    for batch in test_iter:
        text, text_lengths, labels = process_batch(batch, debug=False)
        predictions = model(text, text_lengths)
        loss = criterion(predictions, labels)
        test_loss += loss.item()
        
        probs = torch.softmax(predictions, dim=1)
        preds = torch.argmax(predictions, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probs.extend(probs.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_loss_avg = test_loss / len(test_iter)

try:
    test_probs_array = np.array(test_probs)
    test_labels_bin = label_binarize(test_labels, classes=range(num_classes))
    test_auc = roc_auc_score(test_labels_bin, test_probs_array, average='weighted', multi_class='ovr')
except Exception as e:
    print(f"Warning: Could not calculate AUC-ROC: {e}")
    test_auc = 0.0

baseline_results = {
    'name': 'baseline',
    'dropout': baseline_config['dropout'],
    'grad_clip': baseline_config['grad_clip'],
    'l1_lambda': baseline_config['l1_lambda'],
    'l2_lambda': baseline_config['l2_lambda'],
    'val_acc': best_val_acc,
    'test_acc': test_acc,
    'test_f1': test_f1,
    'test_auc': test_auc
}

print(f"\n>>> Baseline Results:")
print(f"    Validation Acc: {best_val_acc*100:.2f}%")
print(f"    Test Acc: {test_acc*100:.2f}%")
print(f"    Test F1: {test_f1:.4f}")
print(f"    Test AUC-ROC: {test_auc:.4f}")

# Store for next steps
best_grad_clip = baseline_config['grad_clip']  # Will be updated in Step 1
best_dropout = baseline_config['dropout']  # Will be updated in Step 2
best_l1_lambda = baseline_config['l1_lambda']  # Will be updated in Step 3
best_l2_lambda = baseline_config['l2_lambda']  # Will be updated in Step 4


In [ ]:
# ============================================================================
# Regularization Step 1: Gradient Clipping Tuning
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION STEP 1: GRADIENT CLIPPING TUNING")
print("="*80)
print(f"Using baseline settings: dropout={best_dropout}, L1={best_l1_lambda}, L2={best_l2_lambda}")

# Test different gradient clipping values
grad_clip_options = [0.0, 1.0]  # 0.0 = no clipping, 1.0 = clip at 1.0

print(f"\nTesting gradient clipping values: {grad_clip_options}")

step1_results = []

for grad_clip in grad_clip_options:
    print(f"\n{'='*80}")
    print(f"Testing: Gradient Clipping = {grad_clip}")
    print(f"{'='*80}")
    
    # Create model with baseline settings
    model = RNN_Classifier_Aggregation(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=best_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=best_dropout,
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=best_aggregation['method']
    ).to(device)
    
    # Select optimizer
    if best_optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=best_l2_lambda)
    elif best_optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    
    # Training loop
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, labels)
            
            if best_l1_lambda > 0:
                loss = loss + compute_l1_loss(model, best_l1_lambda)
            
            loss.backward()
            
            if grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f'rnn_reg_step1_gradclip{grad_clip}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    step1_results.append({
        'grad_clip': grad_clip,
        'val_acc': best_val_acc
    })
    
    print(f"    Result: Val Acc={best_val_acc*100:.2f}%")

# Find best gradient clipping
best_step1 = max(step1_results, key=lambda x: x['val_acc'])
best_grad_clip = best_step1['grad_clip']

print(f"\n>>> Step 1 Results:")
print(f"{'Grad Clip':<12} {'Val Acc':<10}")
print("-" * 25)
for result in step1_results:
    print(f"{result['grad_clip']:<12} {result['val_acc']*100:<10.2f}%")
print(f"\n>>> Best Gradient Clipping: {best_grad_clip}, Val Acc={best_step1['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Regularization Step 2: Dropout Tuning
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION STEP 2: DROPOUT TUNING")
print("="*80)
print(f"Using best from Step 1: grad_clip={best_grad_clip}")
print(f"Using baseline settings: L1={best_l1_lambda}, L2={best_l2_lambda}")

# Test different dropout values
dropout_options = [0.0, 0.3, 0.5, 0.7]

print(f"\nTesting dropout values: {dropout_options}")

step2_results = []

for dropout_val in dropout_options:
    print(f"\n{'='*80}")
    print(f"Testing: Dropout = {dropout_val}")
    print(f"{'='*80}")
    
    # Create model with best grad_clip and current dropout
    model = RNN_Classifier_Aggregation(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=best_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=dropout_val,
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=best_aggregation['method']
    ).to(device)
    
    # Select optimizer
    if best_optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=best_l2_lambda)
    elif best_optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    
    # Training loop
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, labels)
            
            if best_l1_lambda > 0:
                loss = loss + compute_l1_loss(model, best_l1_lambda)
            
            loss.backward()
            
            if best_grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), best_grad_clip)
            
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f'rnn_reg_step2_dropout{dropout_val}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    step2_results.append({
        'dropout': dropout_val,
        'val_acc': best_val_acc
    })
    
    print(f"    Result: Val Acc={best_val_acc*100:.2f}%")

# Find best dropout
best_step2 = max(step2_results, key=lambda x: x['val_acc'])
best_dropout = best_step2['dropout']

print(f"\n>>> Step 2 Results:")
print(f"{'Dropout':<12} {'Val Acc':<10}")
print("-" * 25)
for result in step2_results:
    print(f"{result['dropout']:<12} {result['val_acc']*100:<10.2f}%")
print(f"\n>>> Best Dropout: {best_dropout}, Val Acc={best_step2['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Regularization Step 3: L1 Regularization Tuning
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION STEP 3: L1 REGULARIZATION TUNING")
print("="*80)
print(f"Using best from Steps 1-2: grad_clip={best_grad_clip}, dropout={best_dropout}")
print(f"Using baseline setting: L2={best_l2_lambda}")

# Test different L1 lambda values
l1_lambda_options = [0.0, 1e-6, 1e-5, 1e-4]

print(f"\nTesting L1 lambda values: {l1_lambda_options}")

step3_results = []

for l1_lambda in l1_lambda_options:
    print(f"\n{'='*80}")
    print(f"Testing: L1 Lambda = {l1_lambda}")
    print(f"{'='*80}")
    
    # Create model with best grad_clip, dropout, and current L1
    model = RNN_Classifier_Aggregation(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=best_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=best_dropout,
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=best_aggregation['method']
    ).to(device)
    
    # Select optimizer
    if best_optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=best_l2_lambda)
    elif best_optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    elif best_optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
    
    # Training loop
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, labels)
            
            if l1_lambda > 0:
                loss = loss + compute_l1_loss(model, l1_lambda)
            
            loss.backward()
            
            if best_grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), best_grad_clip)
            
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f'rnn_reg_step3_l1{l1_lambda}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    step3_results.append({
        'l1_lambda': l1_lambda,
        'val_acc': best_val_acc
    })
    
    print(f"    Result: Val Acc={best_val_acc*100:.2f}%")

# Find best L1 lambda
best_step3 = max(step3_results, key=lambda x: x['val_acc'])
best_l1_lambda = best_step3['l1_lambda']

print(f"\n>>> Step 3 Results:")
print(f"{'L1 Lambda':<12} {'Val Acc':<10}")
print("-" * 25)
for result in step3_results:
    print(f"{result['l1_lambda']:<12.0e} {result['val_acc']*100:<10.2f}%")
print(f"\n>>> Best L1 Lambda: {best_l1_lambda}, Val Acc={best_step3['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Regularization Step 4: L2 Regularization Tuning
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION STEP 4: L2 REGULARIZATION TUNING")
print("="*80)
print(f"Using best from Steps 1-3: grad_clip={best_grad_clip}, dropout={best_dropout}, L1={best_l1_lambda}")

# Test different L2 lambda values (via weight_decay)
l2_lambda_options = [0.0, 1e-5, 1e-4, 1e-3]

print(f"\nTesting L2 lambda values: {l2_lambda_options}")

step4_results = []

for l2_lambda in l2_lambda_options:
    print(f"\n{'='*80}")
    print(f"Testing: L2 Lambda = {l2_lambda}")
    print(f"{'='*80}")
    
    # Create model with best grad_clip, dropout, L1, and current L2
    model = RNN_Classifier_Aggregation(
        vocab_size=fasttext_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=best_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=best_dropout,
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=best_aggregation['method']
    ).to(device)
    
    # Select optimizer with L2 regularization (weight_decay)
    if best_optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=l2_lambda)
    elif best_optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=l2_lambda)
    elif best_optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=l2_lambda)
    elif best_optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=l2_lambda)
    
    # Training loop
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(MAX_EPOCHS):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, labels)
            
            if best_l1_lambda > 0:
                loss = loss + compute_l1_loss(model, best_l1_lambda)
            
            loss.backward()
            
            if best_grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), best_grad_clip)
            
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds)
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in val_iter:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f'rnn_reg_step4_l2{l2_lambda}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")
    
    step4_results.append({
        'l2_lambda': l2_lambda,
        'val_acc': best_val_acc
    })
    
    print(f"    Result: Val Acc={best_val_acc*100:.2f}%")

# Find best L2 lambda
best_step4 = max(step4_results, key=lambda x: x['val_acc'])
best_l2_lambda = best_step4['l2_lambda']

print(f"\n>>> Step 4 Results:")
print(f"{'L2 Lambda':<12} {'Val Acc':<10}")
print("-" * 25)
for result in step4_results:
    print(f"{result['l2_lambda']:<12.0e} {result['val_acc']*100:<10.2f}%")
print(f"\n>>> Best L2 Lambda: {best_l2_lambda}, Val Acc={best_step4['val_acc']*100:.2f}%")


In [ ]:
# ============================================================================
# Regularization Final: All Best Combined
# ============================================================================
print("\n" + "="*80)
print("REGULARIZATION FINAL: ALL BEST SETTINGS COMBINED")
print("="*80)
print(f"Best settings from all steps:")
print(f"    Gradient Clipping: {best_grad_clip}")
print(f"    Dropout: {best_dropout}")
print(f"    L1 Lambda: {best_l1_lambda}")
print(f"    L2 Lambda: {best_l2_lambda}")

# Create model with all best settings
model = RNN_Classifier_Aggregation(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=best_hidden_dim,
    output_dim=num_classes,
    n_layers=N_LAYERS,
    dropout=best_dropout,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings,
    aggregation=best_aggregation['method']
).to(device)

# Select optimizer with best L2 (weight_decay)
if best_optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
elif best_optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=final_lr, momentum=0.9, weight_decay=best_l2_lambda)
elif best_optimizer == 'RMSprop':
    optimizer = optim.RMSprop(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)
elif best_optimizer == 'Adagrad':
    optimizer = optim.Adagrad(model.parameters(), lr=final_lr, weight_decay=best_l2_lambda)

# Training loop
best_val_acc = 0.0
patience_counter = 0

# Store training history for plotting
train_losses = []
val_accs = []

print(f"\n>>> Training final model with all best regularization settings...")

for epoch in range(MAX_EPOCHS):
    model.train()
    train_loss = 0
    train_preds = []
    train_labels = []
    num_batches = 0
    
    for batch in train_iter:
        num_batches += 1
        text, text_lengths, labels = process_batch(batch, debug=False)
        optimizer.zero_grad()
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions, labels)
        
        if best_l1_lambda > 0:
            loss = loss + compute_l1_loss(model, best_l1_lambda)
        
        loss.backward()
        
        if best_grad_clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), best_grad_clip)
        
        optimizer.step()
        
        train_loss += loss.item()
        preds = torch.argmax(predictions, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
    
    train_acc = accuracy_score(train_labels, train_preds)
    # Calculate average training loss (train_loss is sum over all batches)
    num_train_batches = len(train_labels) // train_iter.batch_size + (1 if len(train_labels) % train_iter.batch_size != 0 else 0)
    train_loss_avg = train_loss / num_train_batches if num_train_batches > 0 else train_loss
    
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for batch in val_iter:
            text, text_lengths, labels = process_batch(batch, debug=False)
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            val_loss += loss.item()
            
            preds = torch.argmax(predictions, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_acc = accuracy_score(val_labels, val_preds)
    
    # Store training history for plotting
    train_losses.append(train_loss_avg)
    val_accs.append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'rnn_reg_final_best.pt')
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"    Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%")
            break
    
    if (epoch + 1) % 10 == 0:
        print(f"    Epoch {epoch+1}: Train Acc={train_acc*100:.2f}%, Val Acc={val_acc*100:.2f}%")

# Evaluate on test set
model.load_state_dict(torch.load('rnn_reg_final_best.pt'))
model.eval()

test_loss = 0
test_preds = []
test_labels = []
test_probs = []

with torch.no_grad():
    for batch in test_iter:
        text, text_lengths, labels = process_batch(batch, debug=False)
        predictions = model(text, text_lengths)
        loss = criterion(predictions, labels)
        test_loss += loss.item()
        
        probs = torch.softmax(predictions, dim=1)
        preds = torch.argmax(predictions, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probs.extend(probs.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_loss_avg = test_loss / len(test_iter)

try:
    test_probs_array = np.array(test_probs)
    test_labels_bin = label_binarize(test_labels, classes=range(num_classes))
    test_auc = roc_auc_score(test_labels_bin, test_probs_array, average='weighted', multi_class='ovr')
except Exception as e:
    print(f"Warning: Could not calculate AUC-ROC: {e}")
    test_auc = 0.0

final_results = {
    'name': 'final_combined',
    'dropout': best_dropout,
    'grad_clip': best_grad_clip,
    'l1_lambda': best_l1_lambda,
    'l2_lambda': best_l2_lambda,
    'val_acc': best_val_acc,
    'test_acc': test_acc,
    'test_f1': test_f1,
    'test_auc': test_auc
}

print(f"\n>>> Final Combined Results:")
print(f"    Configuration:")
print(f"      - Gradient Clipping: {best_grad_clip}")
print(f"      - Dropout: {best_dropout}")
print(f"      - L1 Lambda: {best_l1_lambda}")
print(f"      - L2 Lambda: {best_l2_lambda}")
print(f"    Validation Acc: {best_val_acc*100:.2f}%")
print(f"    Test Acc: {test_acc*100:.2f}%")
print(f"    Test F1: {test_f1:.4f}")
print(f"    Test AUC-ROC: {test_auc:.4f}")

# Compare with baseline
improvement = test_acc - baseline_results['test_acc']
improvement_pct = (improvement / baseline_results['test_acc']) * 100 if baseline_results['test_acc'] > 0 else 0

print(f"\n>>> Comparison with Baseline:")
print(f"    Baseline Test Acc: {baseline_results['test_acc']*100:.2f}%")
print(f"    Final Regularized Test Acc: {test_acc*100:.2f}%")
print(f"    Improvement: {improvement*100:+.2f}% ({improvement_pct:+.2f}% relative)")

# Plot training curves for best configuration and regularization
print(f"\n>>> Plotting training curves for best configuration and regularization...")
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Training Loss vs Epochs
epochs = range(1, len(train_losses) + 1)
ax1.plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2, marker='o', markersize=4)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Training Loss', fontsize=12)
ax1.set_title('Training Loss Curve', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(left=1)

# Plot 2: Validation Accuracy vs Epochs
ax2.plot(epochs, [acc*100 for acc in val_accs], 'r-', label='Validation Accuracy', linewidth=2, marker='s', markersize=4)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(left=1)

plt.tight_layout()
plt.savefig('best_config_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'best_config_training_curves.png'")
plt.show()

print(f"\n{'='*80}")
print("REGULARIZATION TUNING COMPLETE")
print(f"{'='*80}")


In [ ]:
# ============================================================================
# Part 2(e): Topic-wise Accuracy Evaluation on Test Set
# ============================================================================
print("\n" + "="*80)
print("PART 2(e): TOPIC-WISE ACCURACY EVALUATION")
print("="*80)

# Load the best model from regularization tuning
# Use the model from the previous cell (regularization final) if available
# Otherwise, load it from the saved checkpoint

print("\n>>> Using model from regularization tuning...")

# Check if 'model' variable exists from the previous cell (cell 26)
try:
    # Try to use the model from the previous cell
    if 'model' in locals() or 'model' in globals():
        # Verify it's a valid model instance
        if hasattr(model, 'embedding') and hasattr(model, 'eval'):
            final_model = model
            final_model.eval()
            saved_vocab_size = final_model.embedding.weight.shape[0]
            print(f"    ✓ Using existing model from previous cell")
            print(f"    Model vocab size: {saved_vocab_size}")
        else:
            raise AttributeError("Model exists but is not valid")
    else:
        raise NameError("Model variable not found")
except (NameError, AttributeError):
    # Model doesn't exist or is invalid, load from checkpoint
    print("    Model not found in previous cell, loading from checkpoint...")
    try:
        checkpoint = torch.load('weights/rnn_reg_final_best.pt', map_location=device)
    except FileNotFoundError:
        checkpoint = torch.load('rnn_reg_final_best.pt', map_location=device)
    
    # Infer configuration from saved state dict
    saved_vocab_size = checkpoint['embedding.weight'].shape[0]
    saved_hidden_dim = checkpoint['rnn.weight_ih_l0'].shape[0]
    has_attention = 'attention.weight' in checkpoint
    saved_aggregation = 'attention' if has_attention else 'last'
    
    # Recreate model
    final_model = RNN_Classifier_Aggregation(
        vocab_size=saved_vocab_size,
        embedding_dim=embedding_dim,
        hidden_dim=saved_hidden_dim,
        output_dim=num_classes,
        n_layers=N_LAYERS,
        dropout=best_dropout,
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=None,
        aggregation=saved_aggregation
    ).to(device)
    
    final_model.load_state_dict(checkpoint, strict=True)
    final_model.eval()
    print(f"    ✓ Model loaded from checkpoint (vocab_size={saved_vocab_size})")

# Function to evaluate per topic
def evaluate_per_topic(model, iterator, device, max_vocab_size=None):
    """
    Evaluate model performance per topic category on the test set.
    Returns a dictionary with accuracy for each topic.
    
    Args:
        model: The trained model
        iterator: Data iterator for test set
        device: Device to run on
        max_vocab_size: Maximum valid vocabulary size (to clip token indices)
    """
    model.eval()
    
    # Dictionary to store correct and total counts per topic
    topic_correct = defaultdict(int)
    topic_total = defaultdict(int)
    
    # Get label vocabulary for mapping
    label_to_idx = LABEL.vocab.stoi
    idx_to_label = LABEL.vocab.itos
    
    # Get <unk> token index for mapping out-of-range tokens
    unk_idx = TEXT.vocab.stoi.get(TEXT.unk_token, 0)
    
    with torch.no_grad():
        for batch in iterator:
            # Process batch (should be on CPU from the iterator)
            text, text_lengths, labels = process_batch(batch, debug=False)
            
            # Clip token indices to valid range if max_vocab_size is specified
            # This handles cases where the current vocab is larger than the saved model's vocab
            if max_vocab_size is not None:
                # Map any indices >= max_vocab_size to <unk> token
                text = torch.where(text >= max_vocab_size, 
                                 torch.tensor(unk_idx, device=text.device, dtype=text.dtype), 
                                 text)
                # Also ensure no negative indices
                text = torch.clamp(text, min=0)
            
            # Move tensors to the actual device after clipping
            text = text.to(device)
            text_lengths = text_lengths.to(device)
            labels = labels.to(device)
            
            predictions = model(text, text_lengths)
            
            # Get predicted labels
            preds = torch.argmax(predictions, dim=1)
            
            # Convert to numpy for easier processing
            preds_np = preds.cpu().numpy()
            labels_np = labels.cpu().numpy()
            
            # Count correct and total for each topic
            for pred_idx, true_idx in zip(preds_np, labels_np):
                true_label = idx_to_label[true_idx]
                topic_total[true_label] += 1
                
                if pred_idx == true_idx:
                    topic_correct[true_label] += 1
    
    # Calculate accuracy per topic
    topic_accuracies = {}
    for label in sorted(topic_total.keys()):
        if topic_total[label] > 0:
            acc = topic_correct[label] / topic_total[label]
            topic_accuracies[label] = {
                'accuracy': acc,
                'correct': topic_correct[label],
                'total': topic_total[label]
            }
    
    return topic_accuracies

# Evaluate on test set
print("\n>>> Evaluating on test set...")

# Create a CPU iterator to avoid CUDA errors during numericalization with invalid token indices
# We'll process on CPU, clip indices, then move to device
from torchtext import data
cpu_device = torch.device('cpu')
test_iter_cpu = data.BucketIterator(
    test_data,
    batch_size=test_iter.batch_size,
    sort_key=lambda x: len(x.text),
    device=cpu_device,
    sort=False,
    shuffle=False
)

# Pass max_vocab_size to clip token indices to the saved model's vocabulary size
topic_accuracies = evaluate_per_topic(final_model, test_iter_cpu, device, max_vocab_size=saved_vocab_size)

# Display results
print("\n" + "="*80)
print("TOPIC-WISE ACCURACY ON TEST SET")
print("="*80)
print(f"{'Topic':<10} {'Accuracy':<12} {'Correct':<10} {'Total':<10} {'% of Test Set':<15}")
print("-" * 80)

# Calculate total test samples for percentage calculation
total_test_samples = sum(acc['total'] for acc in topic_accuracies.values())

for topic in sorted(topic_accuracies.keys()):
    acc_info = topic_accuracies[topic]
    acc_pct = acc_info['accuracy'] * 100
    correct = acc_info['correct']
    total = acc_info['total']
    pct_of_test = (total / total_test_samples) * 100 if total_test_samples > 0 else 0
    
    print(f"{topic:<10} {acc_pct:<12.2f} {correct:<10} {total:<10} {pct_of_test:<15.2f}")

# Calculate overall accuracy
overall_correct = sum(acc_info['correct'] for acc_info in topic_accuracies.values())
overall_total = sum(acc_info['total'] for acc_info in topic_accuracies.values())
overall_acc = overall_correct / overall_total if overall_total > 0 else 0

print("-" * 80)
print(f"{'OVERALL':<10} {overall_acc*100:<12.2f} {overall_correct:<10} {overall_total:<10} {'100.00':<15}")

# ============================================================================
# Discussion: What may cause differences in accuracies across topics
# ============================================================================
print("\n" + "="*80)
print("DISCUSSION: FACTORS AFFECTING TOPIC-WISE ACCURACY")
print("="*80)

# Get training distribution for comparison
train_label_counts = Counter([ex.label for ex in train_data.examples])
total_train = len(train_data.examples)

print("\n1. CLASS IMBALANCE IN TRAINING DATA:")
print(f"{'Topic':<10} {'Train Count':<15} {'Train %':<12} {'Test Count':<12} {'Test %':<12}")
print("-" * 70)
for topic in sorted(topic_accuracies.keys()):
    train_count = train_label_counts.get(topic, 0)
    train_pct = (train_count / total_train) * 100 if total_train > 0 else 0
    test_count = topic_accuracies[topic]['total']
    test_pct = (test_count / total_test_samples) * 100 if total_test_samples > 0 else 0
    print(f"{topic:<10} {train_count:<15} {train_pct:<12.2f} {test_count:<12} {test_pct:<12.2f}")

print("\n2. KEY OBSERVATIONS AND POTENTIAL CAUSES:")
print("\n   a) Class Imbalance Effect:")
print("      - Topics with fewer training examples (e.g., ABBR with only 1.58% of data)")
print("        may have lower accuracy due to insufficient learning signal")
print("      - The model may be biased toward more frequent classes during training")

print("\n   b) Semantic Complexity:")
print("      - Some topics may have more ambiguous or overlapping characteristics")
print("      - For example, ABBR (abbreviations) might be confused with other categories")
print("        if the context is not clear enough")

print("\n   c) Vocabulary and OOV Rates:")
print("      - Topics with higher OOV rates (as seen in Part 1) may have lower accuracy")
print("      - ABBR had 9.70% OOV rate (highest), which could contribute to lower performance")

print("\n   d) Question Type Characteristics:")
print("      - ENTY (entities) and HUM (humans) are more distinct and may be easier to classify")
print("      - DESC (descriptions) might overlap with other categories semantically")
print("      - NUM (numeric) questions may have distinctive patterns that aid classification")

print("\n   e) Model Capacity and Representation:")
print("      - The RNN may capture certain patterns better than others")
print("      - Aggregation method (attention/mean/max/last) may favor certain topic structures")

print("\n3. RECOMMENDATIONS FOR IMPROVEMENT:")
print("   - Use class weights in loss function to handle imbalance")
print("   - Apply data augmentation or oversampling for minority classes")
print("   - Consider focal loss to focus on hard examples")
print("   - Fine-tune embeddings specifically for underrepresented topics")
print("   - Use ensemble methods combining multiple models")

print("\n" + "="*80)
print("TOPIC-WISE EVALUATION COMPLETE")
print("="*80)


In [ ]:
# ============================================================================
# Part 3.1: biLSTM and biGRU Models
# ============================================================================

print("\n" + "="*80)
print("PART 3.1: biLSTM AND biGRU MODELS")
print("="*80)

# biLSTM Model Class
class BiLSTMClassifier(nn.Module):
    """
    Bidirectional LSTM for topic classification.
    Uses pretrained embeddings (learnable/updated during training) with OOV mitigation.
    Incorporates recurrent computations in both directions and supports multiple layers.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=2, dropout=0.5, padding_idx=0, pretrained_embeddings=None):
        super(BiLSTMClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.bidirectional = True
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # Bidirectional LSTM layer with multiple layers
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        # For bidirectional LSTM, output dimension is hidden_dim * 2
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Get dimensions
        batch_size = embedded.size(0)
        seq_len = embedded.size(1)
        
        # Handle text_lengths
        text_lengths_flat = text_lengths.flatten().cpu().long()
        if len(text_lengths_flat) != batch_size:
            raise ValueError(
                f"text_lengths size mismatch: got {len(text_lengths_flat)} elements, "
                f"expected {batch_size}"
            )
        
        # Clamp lengths
        text_lengths_clamped = torch.clamp(text_lengths_flat, min=1, max=seq_len)
        
        # Pack the padded sequences
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths_clamped, batch_first=True, enforce_sorted=False
        )
        
        # Pass through bidirectional LSTM
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # hidden: [n_layers * 2, batch_size, hidden_dim] (2 for bidirectional)
        # For bidirectional LSTM, we concatenate forward and backward hidden states
        # from the last layer
        forward_hidden = hidden[-2]  # [batch_size, hidden_dim]
        backward_hidden = hidden[-1]  # [batch_size, hidden_dim]
        last_hidden = torch.cat([forward_hidden, backward_hidden], dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        last_hidden = self.dropout(last_hidden)
        
        # Pass through fully connected layer
        output = self.fc(last_hidden)  # [batch_size, output_dim]
        
        return output


# biGRU Model Class
class BiGRUClassifier(nn.Module):
    """
    Bidirectional GRU for topic classification.
    Uses pretrained embeddings (learnable/updated during training) with OOV mitigation.
    Incorporates recurrent computations in both directions and supports multiple layers.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=2, dropout=0.5, padding_idx=0, pretrained_embeddings=None):
        super(BiGRUClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.bidirectional = True
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # Bidirectional GRU layer with multiple layers
        self.gru = nn.GRU(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        # For bidirectional GRU, output dimension is hidden_dim * 2
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Get dimensions
        batch_size = embedded.size(0)
        seq_len = embedded.size(1)
        
        # Handle text_lengths
        text_lengths_flat = text_lengths.flatten().cpu().long()
        if len(text_lengths_flat) != batch_size:
            raise ValueError(
                f"text_lengths size mismatch: got {len(text_lengths_flat)} elements, "
                f"expected {batch_size}"
            )
        
        # Clamp lengths
        text_lengths_clamped = torch.clamp(text_lengths_flat, min=1, max=seq_len)
        
        # Pack the padded sequences
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths_clamped, batch_first=True, enforce_sorted=False
        )
        
        # Pass through bidirectional GRU
        packed_output, hidden = self.gru(packed_embedded)
        # hidden: [n_layers * 2, batch_size, hidden_dim] (2 for bidirectional)
        # For bidirectional GRU, we concatenate forward and backward hidden states
        # from the last layer
        forward_hidden = hidden[-2]  # [batch_size, hidden_dim]
        backward_hidden = hidden[-1]  # [batch_size, hidden_dim]
        last_hidden = torch.cat([forward_hidden, backward_hidden], dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        last_hidden = self.dropout(last_hidden)
        
        # Pass through fully connected layer
        output = self.fc(last_hidden)  # [batch_size, output_dim]
        
        return output

print(">>> Model classes created: BiLSTMClassifier and BiGRUClassifier")


In [ ]:
# Training function with history tracking for plotting curves
def train_model_with_history(model, train_iterator, val_iterator, optimizer, criterion, 
                             n_epochs, device, patience=10, model_name="model"):
    """
    Train the model with early stopping and track training history.
    Returns: model, training_history dictionary
    """
    best_val_acc = 0.0
    best_val_f1 = 0.0
    best_val_auc = 0.0
    patience_counter = 0
    best_model_state = None
    
    # History tracking
    train_losses = []
    val_losses = []
    val_accs = []
    val_f1s = []  # Add F1 tracking
    val_aucs = []  # Add AUROC tracking
    
    print(f"\n>>> Training {model_name}")
    print(f"    Parameters: {count_parameters(model):,}")
    print(f"    Max epochs: {n_epochs}, Patience: {patience}")
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        # Training phase
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iterator:
            text, text_lengths, labels = process_batch(batch, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        avg_train_loss = train_loss / len(train_iterator)
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        val_probs = []  # Add probabilities for AUROC
        
        with torch.no_grad():
            for batch in val_iterator:
                text, text_lengths, labels = process_batch(batch, debug=False)
                predictions = model(text, text_lengths)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                probs = torch.softmax(predictions, dim=1)  # Get probabilities
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_probs.extend(probs.cpu().numpy())  # Store probabilities
        
        avg_val_loss = val_loss / len(val_iterator)
        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')  # Calculate F1
        
        # Calculate AUC-ROC
        try:
            val_probs_array = np.array(val_probs)
            val_labels_bin = label_binarize(val_labels, classes=range(num_classes))
            val_auc = roc_auc_score(val_labels_bin, val_probs_array, average='weighted', multi_class='ovr')
        except Exception as e:
            print(f"Warning: Could not calculate AUC-ROC for {model_name} at epoch {epoch+1}: {e}")
            val_auc = 0.0
        
        # Store history
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accs.append(val_acc)
        val_f1s.append(val_f1)  # Store F1
        val_aucs.append(val_auc)  # Store AUROC
        
        # Early stopping and model saving (using accuracy for early stopping)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_f1 = val_f1
            best_val_auc = val_auc
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
        
        print(f'Epoch: {epoch+1:02}/{n_epochs} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
        print(f'\tTrain Loss: {avg_train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\tVal Loss: {avg_val_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}')
        
        if patience_counter >= patience:
            print(f'\t>>> Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%')
            break
    
    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f'\n>>> Training completed! Best validation accuracy: {best_val_acc*100:.2f}%')
    print(f'    Best validation F1: {best_val_f1:.4f}')
    print(f'    Best validation AUC-ROC: {best_val_auc:.4f}')
    
    # Return history dictionary
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_accs': val_accs,
        'val_f1s': val_f1s,  # Add F1 history
        'val_aucs': val_aucs,  # Add AUROC history
        'best_val_acc': best_val_acc,
        'best_val_f1': best_val_f1,  # Add best F1
        'best_val_auc': best_val_auc,  # Add best AUROC
        'epochs_trained': len(train_losses)
    }
    
    return model, history

print(">>> Training function with history tracking created")


In [ ]:
# ============================================================================
# Train biLSTM Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING biLSTM MODEL")
print("="*80)

# Use best hyperparameters from Part 2 (or reasonable defaults)
# Based on Part 2 results, using: LR=0.0001, Batch=32, Hidden=512, but we'll use Hidden=256 for biLSTM
# since bidirectional doubles the effective hidden size
BILSTM_HIDDEN_DIM = 256  # Effective hidden size will be 512 (256*2) due to bidirectional
BILSTM_N_LAYERS = 2  # Stack multiple layers
BILSTM_DROPOUT = 0.5
BILSTM_BATCH_SIZE = 64
BILSTM_LEARNING_RATE = 0.001
BILSTM_N_EPOCHS = 100
BILSTM_PATIENCE = 10

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Create iterators
train_iter_bilstm = data.BucketIterator(
    train_data,
    batch_size=BILSTM_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_bilstm = data.BucketIterator(
    validation_data,
    batch_size=BILSTM_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_bilstm = data.BucketIterator(
    test_data,
    batch_size=BILSTM_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create biLSTM model
bilstm_model = BiLSTMClassifier(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=BILSTM_HIDDEN_DIM,
    output_dim=num_classes,
    n_layers=BILSTM_N_LAYERS,
    dropout=BILSTM_DROPOUT,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
bilstm_optimizer = optim.Adam(bilstm_model.parameters(), lr=BILSTM_LEARNING_RATE)

print(f"\n>>> biLSTM Model Configuration:")
print(f"    Hidden Dim: {BILSTM_HIDDEN_DIM} (effective: {BILSTM_HIDDEN_DIM * 2} due to bidirectional)")
print(f"    Layers: {BILSTM_N_LAYERS}")
print(f"    Dropout: {BILSTM_DROPOUT}")
print(f"    Learning Rate: {BILSTM_LEARNING_RATE}")
print(f"    Batch Size: {BILSTM_BATCH_SIZE}")
print(f"    Max Epochs: {BILSTM_N_EPOCHS}, Patience: {BILSTM_PATIENCE}")

# Train biLSTM
bilstm_model, bilstm_history = train_model_with_history(
    bilstm_model, train_iter_bilstm, val_iter_bilstm, bilstm_optimizer, criterion,
    BILSTM_N_EPOCHS, device, patience=BILSTM_PATIENCE, model_name="biLSTM"
)

# Save best model
torch.save(bilstm_model.state_dict(), 'weights/bilstm_best.pt')
print(f"\n>>> biLSTM model saved to 'weights/bilstm_best.pt'")


In [ ]:
# ============================================================================
# Train biGRU Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING biGRU MODEL")
print("="*80)

# Use same hyperparameters as biLSTM for fair comparison
BIGRU_HIDDEN_DIM = 256  # Effective hidden size will be 512 (256*2) due to bidirectional
BIGRU_N_LAYERS = 2  # Stack multiple layers
BIGRU_DROPOUT = 0.5
BIGRU_BATCH_SIZE = 64
BIGRU_LEARNING_RATE = 0.001
BIGRU_N_EPOCHS = 100
BIGRU_PATIENCE = 10

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Create iterators
train_iter_bigru = data.BucketIterator(
    train_data,
    batch_size=BIGRU_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_bigru = data.BucketIterator(
    validation_data,
    batch_size=BIGRU_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_bigru = data.BucketIterator(
    test_data,
    batch_size=BIGRU_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create biGRU model
bigru_model = BiGRUClassifier(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=BIGRU_HIDDEN_DIM,
    output_dim=num_classes,
    n_layers=BIGRU_N_LAYERS,
    dropout=BIGRU_DROPOUT,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings
).to(device)

# Loss function and optimizer
bigru_optimizer = optim.Adam(bigru_model.parameters(), lr=BIGRU_LEARNING_RATE)

print(f"\n>>> biGRU Model Configuration:")
print(f"    Hidden Dim: {BIGRU_HIDDEN_DIM} (effective: {BIGRU_HIDDEN_DIM * 2} due to bidirectional)")
print(f"    Layers: {BIGRU_N_LAYERS}")
print(f"    Dropout: {BIGRU_DROPOUT}")
print(f"    Learning Rate: {BIGRU_LEARNING_RATE}")
print(f"    Batch Size: {BIGRU_BATCH_SIZE}")
print(f"    Max Epochs: {BIGRU_N_EPOCHS}, Patience: {BIGRU_PATIENCE}")

# Train biGRU
bigru_model, bigru_history = train_model_with_history(
    bigru_model, train_iter_bigru, val_iter_bigru, bigru_optimizer, criterion,
    BIGRU_N_EPOCHS, device, patience=BIGRU_PATIENCE, model_name="biGRU"
)

# Save best model
torch.save(bigru_model.state_dict(), 'weights/bigru_best.pt')
print(f"\n>>> biGRU model saved to 'weights/bigru_best.pt'")


In [ ]:
# ============================================================================
# Plot Training Curves for biLSTM and biGRU
# ============================================================================

print("\n" + "="*80)
print("PLOTTING TRAINING CURVES")
print("="*80)

import matplotlib.pyplot as plt

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Loss Curves
axes[0, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='s', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Training Loss', fontsize=12)
axes[0, 0].set_title('Training Loss Curves', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Accuracy Curves
axes[0, 1].plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM Val Acc', marker='o', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU Val Acc', marker='s', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Validation Accuracy Curves', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: biLSTM Training and Validation Loss
axes[1, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[1, 0].plot(bilstm_history['val_losses'], label='biLSTM Val Loss', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('biLSTM: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: biGRU Training and Validation Loss
axes[1, 1].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='o', linewidth=2)
axes[1, 1].plot(bigru_history['val_losses'], label='biGRU Val Loss', marker='s', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Loss', fontsize=12)
axes[1, 1].set_title('biGRU: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_1_bilstm_bigru_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_1_bilstm_bigru_training_curves.png'")
plt.show()

# Also create separate plots for Part 3.1 requirement (training loss and validation accuracy)
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Training Loss Curve
ax1.plot(bilstm_history['train_losses'], label='biLSTM', marker='o', linewidth=2)
ax1.plot(bigru_history['train_losses'], label='biGRU', marker='s', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Training Loss', fontsize=12)
ax1.set_title('Training Loss Curve (Part 3.1)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM', marker='o', linewidth=2)
ax2.plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU', marker='s', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (Part 3.1)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_1_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved Part 3.1 curves to 'part3_1_training_curves.png'")
plt.show()


In [ ]:
# ============================================================================
# Evaluate biLSTM and biGRU on Test Set
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION - biLSTM AND biGRU")
print("="*80)

def evaluate_model(model, iterator, criterion, device, model_name):
    """Evaluate model on test set and return metrics"""
    model.eval()
    test_loss = 0
    test_preds = []
    test_labels = []
    test_probs = []
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths, labels = process_batch(batch, debug=False)
            predictions = model(text, text_lengths)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
            
            probs = torch.softmax(predictions, dim=1)
            preds = torch.argmax(predictions, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
            test_probs.extend(probs.cpu().numpy())
    
    avg_test_loss = test_loss / len(iterator)
    test_acc = accuracy_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds, average='weighted')
    
    # Calculate AUC-ROC
    try:
        test_probs_array = np.array(test_probs)
        test_labels_bin = label_binarize(test_labels, classes=range(num_classes))
        test_auc = roc_auc_score(test_labels_bin, test_probs_array, average='weighted', multi_class='ovr')
    except Exception as e:
        print(f"Warning: Could not calculate AUC-ROC for {model_name}: {e}")
        test_auc = 0.0
    
    return avg_test_loss, test_acc, test_f1, test_auc

# Evaluate biLSTM
print("\n>>> Evaluating biLSTM on test set...")
bilstm_test_loss, bilstm_test_acc, bilstm_test_f1, bilstm_test_auc = evaluate_model(
    bilstm_model, test_iter_bilstm, criterion, device, "biLSTM"
)

print(f"\n>>> biLSTM Test Set Results:")
print(f"    Test Loss: {bilstm_test_loss:.4f}")
print(f"    Test Accuracy: {bilstm_test_acc*100:.2f}%")
print(f"    Test F1 Score: {bilstm_test_f1:.4f}")
print(f"    Test AUC-ROC: {bilstm_test_auc:.4f}")

# Evaluate biGRU
print("\n>>> Evaluating biGRU on test set...")
bigru_test_loss, bigru_test_acc, bigru_test_f1, bigru_test_auc = evaluate_model(
    bigru_model, test_iter_bigru, criterion, device, "biGRU"
)

print(f"\n>>> biGRU Test Set Results:")
print(f"    Test Loss: {bigru_test_loss:.4f}")
print(f"    Test Accuracy: {bigru_test_acc*100:.2f}%")
print(f"    Test F1 Score: {bigru_test_f1:.4f}")
print(f"    Test AUC-ROC: {bigru_test_auc:.4f}")

# Summary comparison
print("\n" + "="*80)
print("PART 3.1 SUMMARY - biLSTM vs biGRU")
print("="*80)
print(f"{'Model':<10} {'Test Accuracy':<15} {'Test F1':<12} {'Val Acc (Best)':<15}")
print("-" * 60)
print(f"{'biLSTM':<10} {bilstm_test_acc*100:<15.2f}% {bilstm_test_f1:<12.4f} {bilstm_history['best_val_acc']*100:<15.2f}%")
print(f"{'biGRU':<10} {bigru_test_acc*100:<15.2f}% {bigru_test_f1:<12.4f} {bigru_history['best_val_acc']*100:<15.2f}%")
print("="*80)

print("\n>>> Part 3.1 Complete!")
print("    - biLSTM and biGRU models trained with bidirectional and multiple layers")
print("    - Training curves plotted and saved")
print("    - Test set accuracies reported above")


In [ ]:
# ============================================================================
# Part 3.2: Convolutional Neural Network (CNN) Model
# ============================================================================
# Replace simple RNN with CNN to produce sentence representations and perform topic classification

print("\n" + "="*80)
print("PART 3.2: CONVOLUTIONAL NEURAL NETWORK (CNN) MODEL")
print("="*80)

class CNNClassifier(nn.Module):
    """
    CNN for text classification using multiple filter sizes (n-gram features)
    Uses convolutional layers to extract local features and max pooling to create sentence representation
    """
    def __init__(self, vocab_size, embedding_dim, output_dim, 
                 num_filters=100, filter_sizes=[3, 4, 5], dropout=0.5, 
                 padding_idx=0, pretrained_embeddings=None):
        super(CNNClassifier, self).__init__()
        
        self.embedding_dim = embedding_dim
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # Convolutional layers with different filter sizes
        # Each filter size captures different n-gram patterns (e.g., 3-grams, 4-grams, 5-grams)
        # Use (filter_size - 1) // 2 for "same" padding (consistent with hybrid model)
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels=embedding_dim,
                out_channels=num_filters,
                kernel_size=filter_size,
                padding=(filter_size - 1) // 2  # Proper "same" padding
            )
            for filter_size in filter_sizes
        ])
        
        # Dropout layers for regularization
        self.conv_dropout = nn.Dropout(dropout)  # Dropout after CNN conv outputs
        self.dropout = nn.Dropout(dropout)  # Dropout before FC layer
        
        # Fully connected output layer
        # Input size is num_filters * len(filter_sizes) (concatenated features from all filters)
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size] (not used in CNN, but kept for interface compatibility)
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # CNN expects input as [batch_size, channels, seq_len]
        # So we need to transpose: [batch_size, embedding_dim, seq_len]
        embedded = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        
        # Apply each convolutional filter and max pooling
        conv_outputs = []
        for conv in self.convs:
            # Convolution: [batch_size, num_filters, seq_len]
            conv_out = conv(embedded)
            # Apply ReLU activation
            conv_out = torch.relu(conv_out)
            # Apply dropout to conv outputs
            conv_out = self.conv_dropout(conv_out)
            conv_outputs.append(conv_out)
        
        # Ensure all conv outputs have the same sequence length (consistent with hybrid model)
        # Different filter sizes with padding can produce slightly different lengths
        min_seq_len = min(conv_out.size(2) for conv_out in conv_outputs)
        conv_outputs = [conv_out[:, :, :min_seq_len] for conv_out in conv_outputs]
        
        # Apply max pooling over time dimension (seq_len)
        # This extracts the most important feature for each filter
        pooled_outputs = []
        for conv_out in conv_outputs:
            pooled = torch.max(conv_out, dim=2)[0]  # [batch_size, num_filters]
            pooled_outputs.append(pooled)
        
        # Concatenate features from all filter sizes
        # This combines information from different n-gram patterns
        concatenated = torch.cat(pooled_outputs, dim=1)  # [batch_size, num_filters * len(filter_sizes)]
        
        # Apply dropout before FC layer
        concatenated = self.dropout(concatenated)
        
        # Pass through fully connected layer
        output = self.fc(concatenated)  # [batch_size, output_dim]
        
        return output

print(">>> CNNClassifier model class created")
print("    - Uses multiple convolutional filters with different sizes (3, 4, 5)")
print("    - Captures n-gram features at different scales")
print("    - Max pooling to create sentence representation")
print("    - Concatenates features from all filter sizes")



In [ ]:
# ============================================================================
# Train CNN Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING CNN MODEL")
print("="*80)

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameters for CNN
CNN_NUM_FILTERS = 100  # Number of filters per filter size
CNN_FILTER_SIZES = [3, 4, 5]  # Different n-gram sizes to capture
CNN_DROPOUT = 0.5
CNN_BATCH_SIZE = 64
CNN_LEARNING_RATE = 0.001
CNN_N_EPOCHS = 100
CNN_PATIENCE = 10

# Create data iterators
train_iter_cnn = data.BucketIterator(
    train_data,
    batch_size=CNN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_cnn = data.BucketIterator(
    validation_data,
    batch_size=CNN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_cnn = data.BucketIterator(
    test_data,
    batch_size=CNN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create CNN model
cnn_model = CNNClassifier(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    output_dim=num_classes,
    num_filters=CNN_NUM_FILTERS,
    filter_sizes=CNN_FILTER_SIZES,
    dropout=CNN_DROPOUT,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=CNN_LEARNING_RATE)

print(f"\n>>> CNN Model Configuration:")
print(f"    Number of Filters: {CNN_NUM_FILTERS} per filter size")
print(f"    Filter Sizes: {CNN_FILTER_SIZES} (captures {', '.join([f'{s}-grams' for s in CNN_FILTER_SIZES])})")
print(f"    Total Feature Size: {CNN_NUM_FILTERS * len(CNN_FILTER_SIZES)}")
print(f"    Dropout: {CNN_DROPOUT}")
print(f"    Learning Rate: {CNN_LEARNING_RATE}")
print(f"    Batch Size: {CNN_BATCH_SIZE}")
print(f"    Max Epochs: {CNN_N_EPOCHS}, Patience: {CNN_PATIENCE}")

# Train CNN model
cnn_model, cnn_history = train_model_with_history(
    cnn_model, train_iter_cnn, val_iter_cnn, cnn_optimizer, criterion,
    CNN_N_EPOCHS, device, patience=CNN_PATIENCE, model_name="CNN"
)

# Save best model
torch.save(cnn_model.state_dict(), 'weights/cnn_best.pt')
print(f"\n>>> CNN model saved to 'weights/cnn_best.pt'")



In [ ]:
# ============================================================================
# Evaluate CNN Model on Test Set
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION - CNN")
print("="*80)

# Evaluate CNN
print("\n>>> Evaluating CNN on test set...")
cnn_test_loss, cnn_test_acc, cnn_test_f1, cnn_test_auc = evaluate_model(
    cnn_model, test_iter_cnn, criterion, device, "CNN"
)

print(f"\n>>> CNN Test Set Results:")
print(f"    Test Loss: {cnn_test_loss:.4f}")
print(f"    Test Accuracy: {cnn_test_acc*100:.2f}%")
print(f"    Test F1 Score: {cnn_test_f1:.4f}")
print(f"    Test AUC-ROC: {cnn_test_auc:.4f}")



In [ ]:
# ============================================================================
# Part 3.2 Summary - CNN Model
# ============================================================================

print("\n" + "="*80)
print("PART 3.2 SUMMARY - CNN MODEL")
print("="*80)
print(f"{'Model':<15} {'Val Acc':<12} {'Val F1':<12} {'Val AUC':<12} {'Test Acc':<12} {'Test F1':<12} {'Test AUC':<12}")
print("-" * 90)
print(f"{'CNN':<15} {cnn_history['best_val_acc']*100:<12.2f}% {cnn_history['best_val_f1']:<12.4f} {cnn_history['best_val_auc']:<12.4f} {cnn_test_acc*100:<12.2f}% {cnn_test_f1:<12.4f} {cnn_test_auc:<12.4f}")
print("="*90)

print("\n>>> Part 3.2 Complete!")
print("    - CNN model implemented with multiple filter sizes (3, 4, 5)")
print("    - Captures n-gram features at different scales")
print("    - Max pooling to create sentence representation")
print("    - All metrics (Accuracy, F1, AUC-ROC) reported for validation and test sets")



In [ ]:
# ============================================================================
# Plot Training Curves for CNN
# ============================================================================

print("\n" + "="*80)
print("PLOTTING TRAINING CURVES - CNN")
print("="*80)

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Loss Curves (Comparison: biLSTM, biGRU, CNN)
axes[0, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='s', linewidth=2)
axes[0, 0].plot(cnn_history['train_losses'], label='CNN Train Loss', marker='^', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Training Loss', fontsize=12)
axes[0, 0].set_title('Training Loss Curves (Part 3.1 & 3.2)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Accuracy Curves (Comparison: biLSTM, biGRU, CNN)
axes[0, 1].plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM Val Acc', marker='o', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU Val Acc', marker='s', linewidth=2)
axes[0, 1].plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='^', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Validation Accuracy Curves (Part 3.1 & 3.2)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: CNN Training and Validation Loss
axes[1, 0].plot(cnn_history['train_losses'], label='CNN Train Loss', marker='o', linewidth=2)
axes[1, 0].plot(cnn_history['val_losses'], label='CNN Val Loss', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('CNN: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: CNN Training and Validation Accuracy
axes[1, 1].plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='o', linewidth=2, color='green')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[1, 1].set_title('CNN: Validation Accuracy Curve', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_2_cnn_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_2_cnn_training_curves.png'")
plt.show()  # Display the plot in notebook

# Also create separate plots for Part 3.2 requirement (training loss and validation accuracy)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Curve
ax1.plot(cnn_history['train_losses'], label='CNN Train Loss', marker='o', linewidth=2, color='blue')
ax1.plot(cnn_history['val_losses'], label='CNN Val Loss', marker='s', linewidth=2, color='red')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss Curve (Part 3.2)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='o', linewidth=2, color='green')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (Part 3.2)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_2_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved Part 3.2 curves to 'part3_2_training_curves.png'")
plt.show()  # Display the plot in notebook

print("\n>>> Training curves plotted and saved for CNN model")



In [ ]:
# ============================================================================
# Part 3.1 & 3.2 Comparison: BiLSTM vs BiGRU vs CNN
# ============================================================================

print("\n" + "="*80)
print("COMPARISON: biLSTM vs biGRU vs CNN")
print("="*80)
print(f"{'Model':<15} {'Val Acc':<12} {'Val F1':<12} {'Val AUC':<12} {'Test Acc':<12} {'Test F1':<12} {'Test AUC':<12}")
print("-" * 90)
print(f"{'biLSTM':<15} {bilstm_history['best_val_acc']*100:<12.2f}% {bilstm_history['best_val_f1']:<12.4f} {bilstm_history['best_val_auc']:<12.4f} {bilstm_test_acc*100:<12.2f}% {bilstm_test_f1:<12.4f} {bilstm_test_auc:<12.4f}")
print(f"{'biGRU':<15} {bigru_history['best_val_acc']*100:<12.2f}% {bigru_history['best_val_f1']:<12.4f} {bigru_history['best_val_auc']:<12.4f} {bigru_test_acc*100:<12.2f}% {bigru_test_f1:<12.4f} {bigru_test_auc:<12.4f}")
print(f"{'CNN':<15} {cnn_history['best_val_acc']*100:<12.2f}% {cnn_history['best_val_f1']:<12.4f} {cnn_history['best_val_auc']:<12.4f} {cnn_test_acc*100:<12.2f}% {cnn_test_f1:<12.4f} {cnn_test_auc:<12.4f}")
print("="*90)

# Find best model based on test accuracy
models_comparison = {
    'biLSTM': {
        'val_acc': bilstm_history['best_val_acc'],
        'val_f1': bilstm_history['best_val_f1'],
        'val_auc': bilstm_history['best_val_auc'],
        'test_acc': bilstm_test_acc,
        'test_f1': bilstm_test_f1,
        'test_auc': bilstm_test_auc
    },
    'biGRU': {
        'val_acc': bigru_history['best_val_acc'],
        'val_f1': bigru_history['best_val_f1'],
        'val_auc': bigru_history['best_val_auc'],
        'test_acc': bigru_test_acc,
        'test_f1': bigru_test_f1,
        'test_auc': bigru_test_auc
    },
    'CNN': {
        'val_acc': cnn_history['best_val_acc'],
        'val_f1': cnn_history['best_val_f1'],
        'val_auc': cnn_history['best_val_auc'],
        'test_acc': cnn_test_acc,
        'test_f1': cnn_test_f1,
        'test_auc': cnn_test_auc
    }
}

best_model_test_acc = max(models_comparison.items(), key=lambda x: x[1]['test_acc'])
best_model_val_acc = max(models_comparison.items(), key=lambda x: x[1]['val_acc'])

print(f"\n>>> Best Model by Test Accuracy: {best_model_test_acc[0]}")
print(f"    Test Accuracy: {best_model_test_acc[1]['test_acc']*100:.2f}%")
print(f"    Test F1: {best_model_test_acc[1]['test_f1']:.4f}")
print(f"    Test AUC-ROC: {best_model_test_acc[1]['test_auc']:.4f}")

print(f"\n>>> Best Model by Validation Accuracy: {best_model_val_acc[0]}")
print(f"    Validation Accuracy: {best_model_val_acc[1]['val_acc']*100:.2f}%")
print(f"    Validation F1: {best_model_val_acc[1]['val_f1']:.4f}")
print(f"    Validation AUC-ROC: {best_model_val_acc[1]['val_auc']:.4f}")

print("\n>>> Model Architecture Comparison:")
print("    - biLSTM: Bidirectional LSTM with 2 layers, captures sequential dependencies in both directions")
print("    - biGRU: Bidirectional GRU with 2 layers, similar to biLSTM but with fewer parameters")
print("    - CNN: Convolutional layers with multiple filter sizes (3, 4, 5), captures n-gram patterns")



In [ ]:
# # c) 

# 1: (hybrid model) rnn + attention layer
# 2: (hybrid model) cnn + bilstm + attention layer
# 3: (hybrid model) bigru + pretrained transformer (berto)

In [ ]:
# ============================================================================
# Part 3.3: RNN + Attention Layer Model
# ============================================================================
# This implements a more sophisticated attention mechanism (additive/Bahdanau-style)
# compared to the simple linear attention used in Part 2

print("\n" + "="*80)
print("PART 3.3: RNN + ATTENTION LAYER MODEL")
print("="*80)

class RNNWithAttentionClassifier(nn.Module):
    """
    RNN with Additive Attention Mechanism (Bahdanau-style)
    This is a more sophisticated attention mechanism than simple linear attention.
    Uses a feed-forward network to compute attention scores.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=2, dropout=0.5, padding_idx=0, pretrained_embeddings=None,
                 attention_dim=None):
        super(RNNWithAttentionClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.attention_dim = attention_dim if attention_dim else hidden_dim
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # Bidirectional LSTM layer (using LSTM for better performance)
        self.rnn = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=True,  # Bidirectional for better context
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Additive Attention Mechanism (Bahdanau-style)
        # Attention uses a feed-forward network instead of simple linear layer
        # This is more expressive and can learn complex attention patterns
        self.attention_linear1 = nn.Linear(hidden_dim * 2, self.attention_dim)  # *2 for bidirectional
        self.attention_linear2 = nn.Linear(self.attention_dim, 1)
        self.tanh = nn.Tanh()
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        # Input is hidden_dim * 2 (bidirectional) after attention
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Pack sequences for efficient RNN processing
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        # Pass through bidirectional LSTM
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # hidden: [n_layers * 2, batch_size, hidden_dim] (2 for bidirectional)
        
        # Unpack the sequences
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True
        )
        # output: [batch_size, seq_len, hidden_dim * 2]
        
        # Apply Additive Attention Mechanism
        # Step 1: Compute attention scores using feed-forward network
        # This is more sophisticated than simple linear attention
        attention_scores = self.attention_linear1(output)  # [batch_size, seq_len, attention_dim]
        attention_scores = self.tanh(attention_scores)  # Apply tanh activation
        attention_scores = self.attention_linear2(attention_scores).squeeze(2)  # [batch_size, seq_len]
        
        # Step 2: Mask padding positions
        batch_size, seq_len = output.size(0), output.size(1)
        mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths.unsqueeze(1)
        attention_scores = attention_scores.masked_fill(~mask, float('-inf'))
        
        # Step 3: Apply softmax to get attention weights
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(2)  # [batch_size, seq_len, 1]
        
        # Step 4: Compute weighted sum of RNN outputs
        # This gives us a context vector that focuses on important parts of the sequence
        context_vector = torch.sum(attention_weights * output, dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        context_vector = self.dropout(context_vector)
        
        # Pass through fully connected layer
        output = self.fc(context_vector)  # [batch_size, output_dim]
        
        return output

print(">>> RNNWithAttentionClassifier model class created")
print("    - Uses bidirectional LSTM for encoding")
print("    - Implements additive attention mechanism (Bahdanau-style)")
print("    - More sophisticated than simple linear attention")



In [ ]:
# ============================================================================
# Train RNN + Attention Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING RNN + ATTENTION MODEL")
print("="*80)

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameters (using similar settings to BiLSTM/BiGRU for fair comparison)
RNN_ATTN_HIDDEN_DIM = 256  # Effective hidden size will be 512 (256*2) due to bidirectional
RNN_ATTN_N_LAYERS = 2
RNN_ATTN_DROPOUT = 0.5
RNN_ATTN_BATCH_SIZE = 64
RNN_ATTN_LEARNING_RATE = 0.001
RNN_ATTN_N_EPOCHS = 100
RNN_ATTN_PATIENCE = 10
RNN_ATTN_ATTENTION_DIM = 256  # Dimension for attention mechanism

# Create data iterators
train_iter_rnn_attn = data.BucketIterator(
    train_data,
    batch_size=RNN_ATTN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_rnn_attn = data.BucketIterator(
    validation_data,
    batch_size=RNN_ATTN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_rnn_attn = data.BucketIterator(
    test_data,
    batch_size=RNN_ATTN_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create RNN + Attention model
rnn_attn_model = RNNWithAttentionClassifier(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=RNN_ATTN_HIDDEN_DIM,
    output_dim=num_classes,
    n_layers=RNN_ATTN_N_LAYERS,
    dropout=RNN_ATTN_DROPOUT,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings,
    attention_dim=RNN_ATTN_ATTENTION_DIM
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
rnn_attn_optimizer = optim.Adam(rnn_attn_model.parameters(), lr=RNN_ATTN_LEARNING_RATE)

print(f"\n>>> RNN + Attention Model Configuration:")
print(f"    Hidden Dim: {RNN_ATTN_HIDDEN_DIM} (effective: {RNN_ATTN_HIDDEN_DIM * 2} due to bidirectional)")
print(f"    Layers: {RNN_ATTN_N_LAYERS}")
print(f"    Dropout: {RNN_ATTN_DROPOUT}")
print(f"    Attention Dim: {RNN_ATTN_ATTENTION_DIM}")
print(f"    Learning Rate: {RNN_ATTN_LEARNING_RATE}")
print(f"    Batch Size: {RNN_ATTN_BATCH_SIZE}")
print(f"    Max Epochs: {RNN_ATTN_N_EPOCHS}, Patience: {RNN_ATTN_PATIENCE}")

# Train RNN + Attention model
rnn_attn_model, rnn_attn_history = train_model_with_history(
    rnn_attn_model, train_iter_rnn_attn, val_iter_rnn_attn, rnn_attn_optimizer, criterion,
    RNN_ATTN_N_EPOCHS, device, patience=RNN_ATTN_PATIENCE, model_name="RNN+Attention"
)

# Save best model
torch.save(rnn_attn_model.state_dict(), 'weights/rnn_attention_best.pt')
print(f"\n>>> RNN + Attention model saved to 'weights/rnn_attention_best.pt'")



In [ ]:
# ============================================================================
# Evaluate RNN + Attention Model on Test Set
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION - RNN + ATTENTION")
print("="*80)

# Evaluate RNN + Attention
print("\n>>> Evaluating RNN + Attention on test set...")
rnn_attn_test_loss, rnn_attn_test_acc, rnn_attn_test_f1, rnn_attn_test_auc = evaluate_model(
    rnn_attn_model, test_iter_rnn_attn, criterion, device, "RNN+Attention"
)

print(f"\n>>> RNN + Attention Test Set Results:")
print(f"    Test Loss: {rnn_attn_test_loss:.4f}")
print(f"    Test Accuracy: {rnn_attn_test_acc*100:.2f}%")
print(f"    Test F1 Score: {rnn_attn_test_f1:.4f}")
print(f"    Test AUC-ROC: {rnn_attn_test_auc:.4f}")



In [ ]:
# ============================================================================
# Part 3.3 Summary - Compare RNN+Attention with BiLSTM and BiGRU
# ============================================================================

print("\n" + "="*80)
print("PART 3.3 SUMMARY - RNN+ATTENTION vs biLSTM vs biGRU vs CNN")
print("="*80)
print(f"{'Model':<15} {'Val Acc':<12} {'Val F1':<12} {'Val AUC':<12} {'Test Acc':<12} {'Test F1':<12} {'Test AUC':<12}")
print("-" * 90)
print(f"{'biLSTM':<15} {bilstm_history['best_val_acc']*100:<12.2f}% {bilstm_history['best_val_f1']:<12.4f} {bilstm_history['best_val_auc']:<12.4f} {bilstm_test_acc*100:<12.2f}% {bilstm_test_f1:<12.4f} {bilstm_test_auc:<12.4f}")
print(f"{'biGRU':<15} {bigru_history['best_val_acc']*100:<12.2f}% {bigru_history['best_val_f1']:<12.4f} {bigru_history['best_val_auc']:<12.4f} {bigru_test_acc*100:<12.2f}% {bigru_test_f1:<12.4f} {bigru_test_auc:<12.4f}")
print(f"{'CNN':<15} {cnn_history['best_val_acc']*100:<12.2f}% {cnn_history['best_val_f1']:<12.4f} {cnn_history['best_val_auc']:<12.4f} {cnn_test_acc*100:<12.2f}% {cnn_test_f1:<12.4f} {cnn_test_auc:<12.4f}")
print(f"{'RNN+Attention':<15} {rnn_attn_history['best_val_acc']*100:<12.2f}% {rnn_attn_history['best_val_f1']:<12.4f} {rnn_attn_history['best_val_auc']:<12.4f} {rnn_attn_test_acc*100:<12.2f}% {rnn_attn_test_f1:<12.4f} {rnn_attn_test_auc:<12.4f}")
print("="*90)

print("\n>>> Part 3.3 Complete!")
print("    - RNN + Attention model implemented with additive attention mechanism")
print("    - More sophisticated than simple linear attention (uses feed-forward network)")
print("    - Comparison with BiLSTM, BiGRU, and CNN models shown above")
print("    - All metrics (Accuracy, F1, AUC-ROC) reported for validation and test sets")



In [ ]:
# ============================================================================
# Plot Training Curves for RNN + Attention
# ============================================================================

print("\n" + "="*80)
print("PLOTTING TRAINING CURVES - RNN + ATTENTION")
print("="*80)

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Loss Curves (Comparison: biLSTM, biGRU, CNN, RNN+Attention)
axes[0, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='s', linewidth=2)
axes[0, 0].plot(cnn_history['train_losses'], label='CNN Train Loss', marker='^', linewidth=2)
axes[0, 0].plot(rnn_attn_history['train_losses'], label='RNN+Attention Train Loss', marker='d', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Training Loss', fontsize=12)
axes[0, 0].set_title('Training Loss Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Accuracy Curves (Comparison: biLSTM, biGRU, CNN, RNN+Attention)
axes[0, 1].plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM Val Acc', marker='o', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU Val Acc', marker='s', linewidth=2)
axes[0, 1].plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='^', linewidth=2)
axes[0, 1].plot([acc*100 for acc in rnn_attn_history['val_accs']], label='RNN+Attention Val Acc', marker='d', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Validation Accuracy Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: RNN+Attention Training and Validation Loss
axes[1, 0].plot(rnn_attn_history['train_losses'], label='RNN+Attention Train Loss', marker='o', linewidth=2)
axes[1, 0].plot(rnn_attn_history['val_losses'], label='RNN+Attention Val Loss', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('RNN+Attention: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: RNN+Attention Validation Accuracy
axes[1, 1].plot([acc*100 for acc in rnn_attn_history['val_accs']], label='RNN+Attention Val Acc', marker='o', linewidth=2, color='purple')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[1, 1].set_title('RNN+Attention: Validation Accuracy Curve', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_rnn_attention_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_3_rnn_attention_training_curves.png'")
plt.show()  # Display the plot in notebook

# Also create separate plots for Part 3.3 requirement (training loss and validation accuracy)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Curve
ax1.plot(rnn_attn_history['train_losses'], label='RNN+Attention Train Loss', marker='o', linewidth=2, color='blue')
ax1.plot(rnn_attn_history['val_losses'], label='RNN+Attention Val Loss', marker='s', linewidth=2, color='red')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss Curve (Part 3.3)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in rnn_attn_history['val_accs']], label='RNN+Attention Val Acc', marker='o', linewidth=2, color='purple')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (Part 3.3)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved Part 3.3 curves to 'part3_3_training_curves.png'")
plt.show()  # Display the plot in notebook

print("\n>>> Training curves plotted and saved for RNN + Attention model")



In [ ]:
# ============================================================================
# Part 3.3: CNN + BiLSTM + Attention Model
# ============================================================================
# Hybrid model combining CNN, BiLSTM, and Attention mechanisms

print("\n" + "="*80)
print("PART 3.3: CNN + biLSTM + ATTENTION MODEL")
print("="*80)

class CNNBiLSTMAttentionClassifier(nn.Module):
    """
    Modified Hybrid model with label-specific representations:
    - CNN: Extracts n-gram features and produces label-specific latent representations
    - BiLSTM: Processes label-specific representations instead of full sentence
    - Attention: Focuses on relevant label-specific features
    """
    def __init__(self, vocab_size, embedding_dim, output_dim, 
                 num_filters=100, filter_sizes=[3, 4, 5],
                 hidden_dim=256, n_layers=2, dropout=0.5,
                 padding_idx=0, pretrained_embeddings=None,
                 attention_dim=None, label_latent_dim=None):
        super(CNNBiLSTMAttentionClassifier, self).__init__()
        
        self.embedding_dim = embedding_dim
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_dim = output_dim
        self.attention_dim = attention_dim if attention_dim else hidden_dim
        # Dimension for label-specific latent representations
        self.label_latent_dim = label_latent_dim if label_latent_dim else hidden_dim
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            # Make embeddings learnable (updated during training)
            self.embedding.weight.requires_grad = True
        
        # CNN layers to extract n-gram features
        # Use (filter_size - 1) // 2 for "same" padding
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels=embedding_dim,
                out_channels=num_filters,
                kernel_size=filter_size,
                padding=(filter_size - 1) // 2
            )
            for filter_size in filter_sizes
        ])
        
        # CNN produces label-specific latent representations
        # Max pooling over sequence to get sentence-level CNN features
        cnn_output_dim = num_filters * len(filter_sizes)
        # Project CNN features to label-specific latent space (one per class)
        # This creates label-specific sentence representations
        self.cnn_label_projections = nn.ModuleList([
            nn.Linear(cnn_output_dim, self.label_latent_dim)
            for _ in range(output_dim)
        ])
        
        # Dropout layers for regularization
        self.cnn_dropout = nn.Dropout(dropout)  # Dropout after CNN features
        self.bilstm_dropout = nn.Dropout(dropout)  # Dropout after BiLSTM outputs
        self.dropout = nn.Dropout(dropout)  # Dropout after attention
        
        # Bidirectional LSTM processes label-specific representations
        # Input: label_latent_dim (each label-specific representation)
        # Sequence length: output_dim (one representation per class)
        self.bilstm = nn.LSTM(
            self.label_latent_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Additive Attention Mechanism over label-specific BiLSTM outputs
        self.attention_linear1 = nn.Linear(hidden_dim * 2, self.attention_dim)  # *2 for bidirectional
        self.attention_linear2 = nn.Linear(self.attention_dim, 1)
        self.tanh = nn.Tanh()
        
        # Fully connected output layer
        # Input: hidden_dim * 2 (bidirectional) after attention
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        # text_lengths: [batch_size]
        
        batch_size = text.size(0)
        
        # Step 1: Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Step 2: Apply CNN to extract n-gram features
        # CNN expects [batch_size, channels, seq_len]
        embedded_cnn = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        
        conv_outputs = []
        for conv in self.convs:
            # Convolution: [batch_size, num_filters, seq_len]
            conv_out = conv(embedded_cnn)
            conv_out = torch.relu(conv_out)
            conv_outputs.append(conv_out)
        
        # Ensure all conv outputs have the same sequence length
        min_seq_len = min(conv_out.size(2) for conv_out in conv_outputs)
        conv_outputs = [conv_out[:, :, :min_seq_len] for conv_out in conv_outputs]
        
        # Concatenate CNN features from all filter sizes
        # [batch_size, num_filters * len(filter_sizes), seq_len]
        cnn_features = torch.cat(conv_outputs, dim=1)
        
        # Step 3: Produce label-specific latent representations
        # Max pooling over sequence to get sentence-level CNN features
        # [batch_size, num_filters * len(filter_sizes), seq_len] -> [batch_size, num_filters * len(filter_sizes)]
        cnn_sentence_features = torch.max(cnn_features, dim=2)[0]  # [batch_size, cnn_output_dim]
        
        # Apply dropout to CNN features
        cnn_sentence_features = self.cnn_dropout(cnn_sentence_features)
        
        # Project CNN features to label-specific latent representations
        # Each projection creates a label-specific sentence representation
        label_specific_reprs = []
        for label_proj in self.cnn_label_projections:
            label_repr = label_proj(cnn_sentence_features)  # [batch_size, label_latent_dim]
            label_specific_reprs.append(label_repr)
        
        # Stack label-specific representations to form a sequence
        # [batch_size, output_dim, label_latent_dim]
        label_sequence = torch.stack(label_specific_reprs, dim=1)
        
        # Step 4: Pass label-specific representations through BiLSTM
        # BiLSTM processes the sequence of label-specific representations
        # label_sequence: [batch_size, output_dim, label_latent_dim]
        # Create lengths tensor (all sequences have length = output_dim)
        label_lengths = torch.full((batch_size,), self.output_dim, dtype=torch.long, device=text.device)
        
        # Pack sequences for efficient processing
        packed_label_seq = nn.utils.rnn.pack_padded_sequence(
            label_sequence, label_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        packed_output, (hidden, cell) = self.bilstm(packed_label_seq)
        
        # Unpack sequences
        bilstm_output, output_lengths = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True
        )
        # bilstm_output: [batch_size, output_dim, hidden_dim * 2]
        
        # Apply dropout to BiLSTM outputs
        bilstm_output = self.bilstm_dropout(bilstm_output)
        
        # Step 5: Apply Attention Mechanism over label-specific BiLSTM outputs
        # Compute attention scores using feed-forward network
        attention_scores = self.attention_linear1(bilstm_output)  # [batch_size, output_dim, attention_dim]
        attention_scores = self.tanh(attention_scores)
        attention_scores = self.attention_linear2(attention_scores).squeeze(2)  # [batch_size, output_dim]
        
        # Apply softmax to get attention weights over label-specific representations
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(2)  # [batch_size, output_dim, 1]
        
        # Compute weighted sum of label-specific BiLSTM outputs
        context_vector = torch.sum(attention_weights * bilstm_output, dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        context_vector = self.dropout(context_vector)
        
        # Step 6: Pass through fully connected layer
        output = self.fc(context_vector)  # [batch_size, output_dim]
        
        return output

print(">>> CNNBiLSTMAttentionClassifier model class created")
print("    - Combines CNN (n-gram features), BiLSTM (sequential dependencies), and Attention")
print("    - CNN extracts local patterns, BiLSTM captures long-range dependencies")
print("    - Attention mechanism focuses on important parts of the sequence")



In [ ]:
# ============================================================================
# Train CNN + BiLSTM + Attention Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING CNN + biLSTM + ATTENTION MODEL")
print("="*80)

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameters for CNN + BiLSTM + Attention
HYBRID_NUM_FILTERS = 100
HYBRID_FILTER_SIZES = [3, 4, 5]
HYBRID_HIDDEN_DIM = 256  # Effective hidden size will be 512 (256*2) due to bidirectional
HYBRID_N_LAYERS = 2
HYBRID_DROPOUT = 0.5
HYBRID_BATCH_SIZE = 64
HYBRID_LEARNING_RATE = 0.001
HYBRID_N_EPOCHS = 100
HYBRID_PATIENCE = 10
HYBRID_ATTENTION_DIM = 256

# Create data iterators
train_iter_hybrid = data.BucketIterator(
    train_data,
    batch_size=HYBRID_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_hybrid = data.BucketIterator(
    validation_data,
    batch_size=HYBRID_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_hybrid = data.BucketIterator(
    test_data,
    batch_size=HYBRID_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create CNN + BiLSTM + Attention model
hybrid_model = CNNBiLSTMAttentionClassifier(
    vocab_size=fasttext_vocab_size,
    embedding_dim=embedding_dim,
    output_dim=num_classes,
    num_filters=HYBRID_NUM_FILTERS,
    filter_sizes=HYBRID_FILTER_SIZES,
    hidden_dim=HYBRID_HIDDEN_DIM,
    n_layers=HYBRID_N_LAYERS,
    dropout=HYBRID_DROPOUT,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
    pretrained_embeddings=pretrained_embeddings,
    attention_dim=HYBRID_ATTENTION_DIM
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
hybrid_optimizer = optim.Adam(hybrid_model.parameters(), lr=HYBRID_LEARNING_RATE)

print(f"\n>>> CNN + BiLSTM + Attention Model Configuration:")
print(f"    CNN Filters: {HYBRID_NUM_FILTERS} per filter size")
print(f"    CNN Filter Sizes: {HYBRID_FILTER_SIZES}")
print(f"    BiLSTM Hidden Dim: {HYBRID_HIDDEN_DIM} (effective: {HYBRID_HIDDEN_DIM * 2} due to bidirectional)")
print(f"    BiLSTM Layers: {HYBRID_N_LAYERS}")
print(f"    Attention Dim: {HYBRID_ATTENTION_DIM}")
print(f"    Dropout: {HYBRID_DROPOUT}")
print(f"    Learning Rate: {HYBRID_LEARNING_RATE}")
print(f"    Batch Size: {HYBRID_BATCH_SIZE}")
print(f"    Max Epochs: {HYBRID_N_EPOCHS}, Patience: {HYBRID_PATIENCE}")

# Train hybrid model
hybrid_model, hybrid_history = train_model_with_history(
    hybrid_model, train_iter_hybrid, val_iter_hybrid, hybrid_optimizer, criterion,
    HYBRID_N_EPOCHS, device, patience=HYBRID_PATIENCE, model_name="CNN+BiLSTM+Attention"
)

# Save best model
torch.save(hybrid_model.state_dict(), 'weights/cnn_bilstm_attention_best.pt')
print(f"\n>>> CNN + BiLSTM + Attention model saved to 'weights/cnn_bilstm_attention_best.pt'")



In [ ]:
# ============================================================================
# Evaluate CNN + BiLSTM + Attention Model on Test Set
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION - CNN + biLSTM + ATTENTION")
print("="*80)

# Evaluate hybrid model
print("\n>>> Evaluating CNN + BiLSTM + Attention on test set...")
hybrid_test_loss, hybrid_test_acc, hybrid_test_f1, hybrid_test_auc = evaluate_model(
    hybrid_model, test_iter_hybrid, criterion, device, "CNN+BiLSTM+Attention"
)

print(f"\n>>> CNN + BiLSTM + Attention Test Set Results:")
print(f"    Test Loss: {hybrid_test_loss:.4f}")
print(f"    Test Accuracy: {hybrid_test_acc*100:.2f}%")
print(f"    Test F1 Score: {hybrid_test_f1:.4f}")
print(f"    Test AUC-ROC: {hybrid_test_auc:.4f}")



In [ ]:
# ============================================================================
# Part 3.3 Updated Summary - All Models Including CNN + BiLSTM + Attention
# ============================================================================

print("\n" + "="*80)
print("PART 3.3 SUMMARY - ALL MODELS")
print("="*80)
print(f"{'Model':<20} {'Val Acc':<12} {'Val F1':<12} {'Val AUC':<12} {'Test Acc':<12} {'Test F1':<12} {'Test AUC':<12}")
print("-" * 100)
print(f"{'biLSTM':<20} {bilstm_history['best_val_acc']*100:<12.2f}% {bilstm_history['best_val_f1']:<12.4f} {bilstm_history['best_val_auc']:<12.4f} {bilstm_test_acc*100:<12.2f}% {bilstm_test_f1:<12.4f} {bilstm_test_auc:<12.4f}")
print(f"{'biGRU':<20} {bigru_history['best_val_acc']*100:<12.2f}% {bigru_history['best_val_f1']:<12.4f} {bigru_history['best_val_auc']:<12.4f} {bigru_test_acc*100:<12.2f}% {bigru_test_f1:<12.4f} {bigru_test_auc:<12.4f}")
print(f"{'CNN':<20} {cnn_history['best_val_acc']*100:<12.2f}% {cnn_history['best_val_f1']:<12.4f} {cnn_history['best_val_auc']:<12.4f} {cnn_test_acc*100:<12.2f}% {cnn_test_f1:<12.4f} {cnn_test_auc:<12.4f}")
print(f"{'RNN+Attention':<20} {rnn_attn_history['best_val_acc']*100:<12.2f}% {rnn_attn_history['best_val_f1']:<12.4f} {rnn_attn_history['best_val_auc']:<12.4f} {rnn_attn_test_acc*100:<12.2f}% {rnn_attn_test_f1:<12.4f} {rnn_attn_test_auc:<12.4f}")
print(f"{'CNN+BiLSTM+Attn':<20} {hybrid_history['best_val_acc']*100:<12.2f}% {hybrid_history['best_val_f1']:<12.4f} {hybrid_history['best_val_auc']:<12.4f} {hybrid_test_acc*100:<12.2f}% {hybrid_test_f1:<12.4f} {hybrid_test_auc:<12.4f}")
print("="*100)

# Find best model based on test accuracy
all_models_comparison = {
    'biLSTM': {
        'val_acc': bilstm_history['best_val_acc'],
        'val_f1': bilstm_history['best_val_f1'],
        'val_auc': bilstm_history['best_val_auc'],
        'test_acc': bilstm_test_acc,
        'test_f1': bilstm_test_f1,
        'test_auc': bilstm_test_auc
    },
    'biGRU': {
        'val_acc': bigru_history['best_val_acc'],
        'val_f1': bigru_history['best_val_f1'],
        'val_auc': bigru_history['best_val_auc'],
        'test_acc': bigru_test_acc,
        'test_f1': bigru_test_f1,
        'test_auc': bigru_test_auc
    },
    'CNN': {
        'val_acc': cnn_history['best_val_acc'],
        'val_f1': cnn_history['best_val_f1'],
        'val_auc': cnn_history['best_val_auc'],
        'test_acc': cnn_test_acc,
        'test_f1': cnn_test_f1,
        'test_auc': cnn_test_auc
    },
    'RNN+Attention': {
        'val_acc': rnn_attn_history['best_val_acc'],
        'val_f1': rnn_attn_history['best_val_f1'],
        'val_auc': rnn_attn_history['best_val_auc'],
        'test_acc': rnn_attn_test_acc,
        'test_f1': rnn_attn_test_f1,
        'test_auc': rnn_attn_test_auc
    },
    'CNN+BiLSTM+Attention': {
        'val_acc': hybrid_history['best_val_acc'],
        'val_f1': hybrid_history['best_val_f1'],
        'val_auc': hybrid_history['best_val_auc'],
        'test_acc': hybrid_test_acc,
        'test_f1': hybrid_test_f1,
        'test_auc': hybrid_test_auc
    }
}

best_model_test_acc = max(all_models_comparison.items(), key=lambda x: x[1]['test_acc'])
best_model_val_acc = max(all_models_comparison.items(), key=lambda x: x[1]['val_acc'])

print(f"\n>>> Best Model by Test Accuracy: {best_model_test_acc[0]}")
print(f"    Test Accuracy: {best_model_test_acc[1]['test_acc']*100:.2f}%")
print(f"    Test F1: {best_model_test_acc[1]['test_f1']:.4f}")
print(f"    Test AUC-ROC: {best_model_test_acc[1]['test_auc']:.4f}")

print(f"\n>>> Best Model by Validation Accuracy: {best_model_val_acc[0]}")
print(f"    Validation Accuracy: {best_model_val_acc[1]['val_acc']*100:.2f}%")
print(f"    Validation F1: {best_model_val_acc[1]['val_f1']:.4f}")
print(f"    Validation AUC-ROC: {best_model_val_acc[1]['val_auc']:.4f}")

print("\n>>> Part 3.3 Complete!")
print("    - RNN + Attention: Additive attention mechanism with bidirectional LSTM")
print("    - CNN + BiLSTM + Attention: Hybrid model combining CNN, BiLSTM, and attention")
print("    - All models compared above with validation and test metrics")
print("    - All metrics (Accuracy, F1, AUC-ROC) reported for validation and test sets")



In [ ]:
# ============================================================================
# Plot Training Curves for CNN + BiLSTM + Attention
# ============================================================================

print("\n" + "="*80)
print("PLOTTING TRAINING CURVES - CNN + biLSTM + ATTENTION")
print("="*80)

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Loss Curves (All Models)
axes[0, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='s', linewidth=2)
axes[0, 0].plot(cnn_history['train_losses'], label='CNN Train Loss', marker='^', linewidth=2)
axes[0, 0].plot(rnn_attn_history['train_losses'], label='RNN+Attention Train Loss', marker='d', linewidth=2)
axes[0, 0].plot(hybrid_history['train_losses'], label='CNN+BiLSTM+Attn Train Loss', marker='*', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Training Loss', fontsize=12)
axes[0, 0].set_title('Training Loss Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Accuracy Curves (All Models)
axes[0, 1].plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM Val Acc', marker='o', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU Val Acc', marker='s', linewidth=2)
axes[0, 1].plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='^', linewidth=2)
axes[0, 1].plot([acc*100 for acc in rnn_attn_history['val_accs']], label='RNN+Attention Val Acc', marker='d', linewidth=2)
axes[0, 1].plot([acc*100 for acc in hybrid_history['val_accs']], label='CNN+BiLSTM+Attn Val Acc', marker='*', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Validation Accuracy Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: CNN+BiLSTM+Attention Training and Validation Loss
axes[1, 0].plot(hybrid_history['train_losses'], label='CNN+BiLSTM+Attn Train Loss', marker='o', linewidth=2)
axes[1, 0].plot(hybrid_history['val_losses'], label='CNN+BiLSTM+Attn Val Loss', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('CNN+BiLSTM+Attention: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: CNN+BiLSTM+Attention Validation Accuracy
axes[1, 1].plot([acc*100 for acc in hybrid_history['val_accs']], label='CNN+BiLSTM+Attn Val Acc', marker='o', linewidth=2, color='orange')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[1, 1].set_title('CNN+BiLSTM+Attention: Validation Accuracy Curve', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_cnn_bilstm_attention_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_3_cnn_bilstm_attention_training_curves.png'")
plt.show()  # Display the plot in notebook

# Also create separate plots for Part 3.3 requirement (training loss and validation accuracy)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Curve
ax1.plot(hybrid_history['train_losses'], label='CNN+BiLSTM+Attn Train Loss', marker='o', linewidth=2, color='blue')
ax1.plot(hybrid_history['val_losses'], label='CNN+BiLSTM+Attn Val Loss', marker='s', linewidth=2, color='red')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss Curve (CNN+BiLSTM+Attention)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in hybrid_history['val_accs']], label='CNN+BiLSTM+Attn Val Acc', marker='o', linewidth=2, color='orange')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (CNN+BiLSTM+Attention)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_hybrid_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved Part 3.3 hybrid curves to 'part3_3_hybrid_training_curves.png'")
plt.show()  # Display the plot in notebook

print("\n>>> Training curves plotted and saved for CNN + BiLSTM + Attention model")



In [ ]:
from huggingface_hub import login
login("")


In [ ]:
# ============================================================================
# Part 3.3: RNN + Pretrained BERT Model
# ============================================================================
# Hybrid model combining pretrained BERT embeddings with RNN

print("\n" + "="*80)
print("PART 3.3: RNN + PRETRAINED BERT MODEL")
print("="*80)

# Import transformers for BERT
try:
    from transformers import BertModel, BertTokenizer
    BERT_AVAILABLE = True
    print(">>> Transformers library available")
except ImportError:
    print(">>> Warning: transformers library not found. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
    from transformers import BertModel, BertTokenizer
    BERT_AVAILABLE = True
    print(">>> Transformers library installed")

class RNNBertClassifier(nn.Module):
    """
    RNN with Pretrained BERT embeddings
    Uses BERT to get contextualized embeddings, then passes through BiLSTM with attention
    """
    def __init__(self, output_dim, hidden_dim=256, n_layers=2, dropout=0.5,
                 bert_model_name='distilbert-base-uncased', freeze_bert=False):
        super(RNNBertClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.freeze_bert = freeze_bert
        
        # Load pretrained BERT model and tokenizer
        self.bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        
        # Freeze BERT parameters if specified
        if freeze_bert:
            for param in self.bert_model.parameters():
                param.requires_grad = False
        
        # BERT output dimension (768 for bert-base-uncased)
        bert_output_dim = self.bert_model.config.hidden_size
        
        # Bidirectional LSTM to process BERT embeddings
        self.bilstm = nn.LSTM(
            bert_output_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Additive Attention Mechanism (Bahdanau-style)
        self.attention_linear1 = nn.Linear(hidden_dim * 2, hidden_dim)  # *2 for bidirectional
        self.attention_linear2 = nn.Linear(hidden_dim, 1)
        self.tanh = nn.Tanh()
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text, text_lengths, text_vocab=None):
        # text: [batch_size, seq_len] - token indices from torchtext
        # text_lengths: [batch_size] - actual sequence lengths
        # text_vocab: TEXT.vocab.itos to convert indices to tokens
        
        batch_size = text.size(0)
        seq_len = text.size(1)
        device = text.device
        
        # Convert token indices back to text strings using vocab
        text_list = []
        for i in range(batch_size):
            actual_len = text_lengths[i].item() if isinstance(text_lengths[i], torch.Tensor) else text_lengths[i]
            tokens = []
            for j in range(min(actual_len, seq_len)):
                token_idx = text[i, j].item()
                if text_vocab is not None and token_idx < len(text_vocab):
                    token = text_vocab[token_idx]
                    # Skip special tokens
                    if token not in ['<pad>', '<unk>', '<sos>', '<eos>']:
                        tokens.append(token)
                else:
                    # Fallback if vocab not provided
                    tokens.append(str(token_idx))
            # Join tokens to form sentence
            sentence = " ".join(tokens)
            text_list.append(sentence)
        
        # Tokenize with BERT
        encoded = self.bert_tokenizer(
            text_list,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)
        
        # Get BERT embeddings
        with torch.set_grad_enabled(not self.freeze_bert):
            bert_outputs = self.bert_model(**encoded)
            bert_embeddings = bert_outputs.last_hidden_state  # [batch_size, seq_len, 768]
        
        # Get actual sequence lengths from BERT tokenizer
        bert_lengths = encoded['attention_mask'].sum(dim=1).cpu()


        # Get BERT embeddings
        with torch.set_grad_enabled(not self.freeze_bert):
            bert_outputs = self.bert_model(**encoded)
            bert_embeddings = bert_outputs.last_hidden_state  # [batch_size, seq_len, 768]

        # Align lengths
        bert_lengths = encoded['attention_mask'].sum(dim=1)
        max_len = bert_embeddings.size(1)
        bert_lengths = bert_lengths.clamp(max=max_len).cpu()

        # Pack safely
        packed_bert = nn.utils.rnn.pack_padded_sequence(
        bert_embeddings, bert_lengths, batch_first=True, enforce_sorted=False
        )

        
        # Pack sequences for efficient RNN processing
        packed_bert = nn.utils.rnn.pack_padded_sequence(
            bert_embeddings, bert_lengths, batch_first=True, enforce_sorted=False
        )
        
        # Pass through bidirectional LSTM
        packed_output, (hidden, cell) = self.bilstm(packed_bert)
        
        # Unpack sequences
        bilstm_output, output_lengths = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True
        )
        # bilstm_output: [batch_size, seq_len, hidden_dim * 2]
        
        # Apply Attention Mechanism
        attention_scores = self.attention_linear1(bilstm_output)  # [batch_size, seq_len, hidden_dim]
        attention_scores = self.tanh(attention_scores)
        attention_scores = self.attention_linear2(attention_scores).squeeze(2)  # [batch_size, seq_len]
        
        # Mask padding positions
        batch_size_attn, seq_len_attn = bilstm_output.size(0), bilstm_output.size(1)
        mask = torch.arange(seq_len_attn, device=device).unsqueeze(0) < bert_lengths.unsqueeze(1).to(device)
        attention_scores = attention_scores.masked_fill(~mask, float('-inf'))
        
        # Apply softmax to get attention weights
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(2)  # [batch_size, seq_len, 1]
        
        # Compute weighted sum
        context_vector = torch.sum(attention_weights * bilstm_output, dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        context_vector = self.dropout(context_vector)
        
        # Pass through fully connected layer
        output = self.fc(context_vector)  # [batch_size, output_dim]
        
        return output

print(">>> RNNBertClassifier model class created")
print("    - Uses pretrained BERT (bert-base-uncased) for contextualized embeddings")
print("    - Passes BERT embeddings through bidirectional LSTM")
print("    - Applies attention mechanism to focus on important parts")



In [ ]:
# Helper function to process batch for BERT model
def process_batch_bert(batch, text_vocab, debug=False):
    text, text_lengths = batch.text
    labels = batch.label

    # Convert to proper format
    if isinstance(text, tuple):
        text, text_lengths = text

    # >>> ADD THIS LINE <<<
    text = text.transpose(0, 1)  # (seq_len, batch_size) → (batch_size, seq_len)

    # Ensure text_lengths is 1D
    if text_lengths.dim() > 1:
        text_lengths = text_lengths.squeeze()

    text = text.to(device)
    text_lengths = text_lengths.to(device)
    labels = labels.to(device)

    if debug:
        print(f"DEBUG BERT BATCH - text shape: {text.shape}, text_lengths shape: {text_lengths.shape}, labels shape: {labels.shape}")
    
    return text, text_lengths, labels, text_vocab



In [ ]:
# ============================================================================
# Train RNN + BERT Model
# ============================================================================

print("\n" + "="*80)
print("TRAINING RNN + BERT MODEL")
print("="*80)

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameters for RNN + BERT
BERT_HIDDEN_DIM = 256  # Effective hidden size will be 512 (256*2) due to bidirectional
BERT_N_LAYERS = 2
BERT_DROPOUT = 0.5
BERT_BATCH_SIZE = 32  # Smaller batch size due to BERT's memory requirements
BERT_LEARNING_RATE = 2e-5  # Lower learning rate for fine-tuning BERT
BERT_N_EPOCHS = 50  # Fewer epochs as BERT converges faster
BERT_PATIENCE = 7
BERT_FREEZE = False  # Fine-tune BERT (set to True to freeze BERT)

# Create data iterators
train_iter_bert = data.BucketIterator(
    train_data,
    batch_size=BERT_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

val_iter_bert = data.BucketIterator(
    validation_data,
    batch_size=BERT_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_bert = data.BucketIterator(
    test_data,
    batch_size=BERT_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

# Create RNN + BERT model
bert_model = RNNBertClassifier(
    output_dim=num_classes,
    hidden_dim=BERT_HIDDEN_DIM,
    n_layers=BERT_N_LAYERS,
    dropout=BERT_DROPOUT,
    bert_model_name='bert-base-uncased',
    freeze_bert=BERT_FREEZE
).to(device)

# Loss function and optimizer
# Use different learning rates for BERT and other layers
bert_params = list(bert_model.bert_model.parameters())
other_params = [p for n, p in bert_model.named_parameters() if 'bert_model' not in n]

bert_optimizer = optim.AdamW([
    {'params': bert_params, 'lr': BERT_LEARNING_RATE},
    {'params': other_params, 'lr': BERT_LEARNING_RATE * 10}  # Higher LR for new layers
], weight_decay=0.01)

print(f"\n>>> RNN + BERT Model Configuration:")
print(f"    BERT Model: bert-base-uncased")
print(f"    BERT Frozen: {BERT_FREEZE}")
print(f"    BiLSTM Hidden Dim: {BERT_HIDDEN_DIM} (effective: {BERT_HIDDEN_DIM * 2} due to bidirectional)")
print(f"    BiLSTM Layers: {BERT_N_LAYERS}")
print(f"    Dropout: {BERT_DROPOUT}")
print(f"    Learning Rate: {BERT_LEARNING_RATE} (BERT), {BERT_LEARNING_RATE * 10} (other layers)")
print(f"    Batch Size: {BERT_BATCH_SIZE}")
print(f"    Max Epochs: {BERT_N_EPOCHS}, Patience: {BERT_PATIENCE}")

# Modified training function for BERT
def train_model_with_history_bert(model, train_iterator, val_iterator, optimizer, criterion, 
                                   n_epochs, device, patience=10, model_name="model", text_vocab=None):
    """
    Train the model with early stopping and track training history (BERT version)
    """
    best_val_acc = 0.0
    best_val_f1 = 0.0
    best_val_auc = 0.0
    patience_counter = 0
    best_model_state = None
    
    # History tracking
    train_losses = []
    val_losses = []
    val_accs = []
    val_f1s = []
    val_aucs = []
    
    print(f"\n>>> Training {model_name}")
    print(f"    Parameters: {count_parameters(model):,}")
    print(f"    Max epochs: {n_epochs}, Patience: {patience}")
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        # Training phase
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch in train_iterator:
            text, text_lengths, labels, _ = process_batch_bert(batch, text_vocab, debug=False)
            optimizer.zero_grad()
            predictions = model(text, text_lengths, text_vocab=text_vocab)
            loss = criterion(predictions, labels)
            loss.backward()
            # Gradient clipping for BERT
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item()
            preds = torch.argmax(predictions, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        avg_train_loss = train_loss / len(train_iterator)
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        val_probs = []
        
        with torch.no_grad():
            for batch in val_iterator:
                text, text_lengths, labels, _ = process_batch_bert(batch, text_vocab, debug=False)
                predictions = model(text, text_lengths, text_vocab=text_vocab)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
                
                probs = torch.softmax(predictions, dim=1)
                preds = torch.argmax(predictions, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_probs.extend(probs.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_iterator)
        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        
        # Calculate AUC-ROC
        try:
            val_probs_array = np.array(val_probs)
            val_labels_bin = label_binarize(val_labels, classes=range(num_classes))
            val_auc = roc_auc_score(val_labels_bin, val_probs_array, average='weighted', multi_class='ovr')
        except Exception as e:
            print(f"Warning: Could not calculate AUC-ROC for {model_name} at epoch {epoch+1}: {e}")
            val_auc = 0.0
        
        # Store history
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accs.append(val_acc)
        val_f1s.append(val_f1)
        val_aucs.append(val_auc)
        
        # Early stopping and model saving
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_f1 = val_f1
            best_val_auc = val_auc
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
        
        print(f'Epoch: {epoch+1:02}/{n_epochs} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
        print(f'\tTrain Loss: {avg_train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\tVal Loss: {avg_val_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}')
        
        if patience_counter >= patience:
            print(f'\t>>> Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%')
            break
    
    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f'\n>>> Training completed! Best validation accuracy: {best_val_acc*100:.2f}%')
    print(f'    Best validation F1: {best_val_f1:.4f}')
    print(f'    Best validation AUC-ROC: {best_val_auc:.4f}')
    
    # Return history dictionary
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_accs': val_accs,
        'val_f1s': val_f1s,
        'val_aucs': val_aucs,
        'best_val_acc': best_val_acc,
        'best_val_f1': best_val_f1,
        'best_val_auc': best_val_auc,
        'epochs_trained': len(train_losses)
    }
    
    return model, history

# Train RNN + BERT model
bert_model, bert_history = train_model_with_history_bert(
    bert_model, train_iter_bert, val_iter_bert, bert_optimizer, criterion,
    BERT_N_EPOCHS, device, patience=BERT_PATIENCE, model_name="RNN+BERT", text_vocab=TEXT.vocab.itos
)

# Save best model
torch.save(bert_model.state_dict(), 'weights/rnn_bert_best.pt')
print(f"\n>>> RNN + BERT model saved to 'weights/rnn_bert_best.pt'")



In [ ]:
# ============================================================================
# Evaluate RNN + BERT Model on Test Set
# ============================================================================

print("\n" + "="*80)
print("TEST SET EVALUATION - RNN + BERT")
print("="*80)

# Modified evaluate function for BERT
def evaluate_model_bert(model, iterator, criterion, device, model_name, text_vocab=None):
    """Evaluate model on test set and return metrics (BERT version)"""
    model.eval()
    test_loss = 0
    test_preds = []
    test_labels = []
    test_probs = []
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths, labels, _ = process_batch_bert(batch, text_vocab, debug=False)
            predictions = model(text, text_lengths, text_vocab=text_vocab)
            loss = criterion(predictions, labels)
            test_loss += loss.item()
            
            probs = torch.softmax(predictions, dim=1)
            preds = torch.argmax(predictions, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
            test_probs.extend(probs.cpu().numpy())
    
    avg_test_loss = test_loss / len(iterator)
    test_acc = accuracy_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds, average='weighted')
    
    # Calculate AUC-ROC
    try:
        test_probs_array = np.array(test_probs)
        test_labels_bin = label_binarize(test_labels, classes=range(num_classes))
        test_auc = roc_auc_score(test_labels_bin, test_probs_array, average='weighted', multi_class='ovr')
    except Exception as e:
        print(f"Warning: Could not calculate AUC-ROC for {model_name}: {e}")
        test_auc = 0.0
    
    return avg_test_loss, test_acc, test_f1, test_auc

# Evaluate RNN + BERT
print("\n>>> Evaluating RNN + BERT on test set...")
bert_test_loss, bert_test_acc, bert_test_f1, bert_test_auc = evaluate_model_bert(
    bert_model, test_iter_bert, criterion, device, "RNN+BERT", text_vocab=TEXT.vocab.itos
)

print(f"\n>>> RNN + BERT Test Set Results:")
print(f"    Test Loss: {bert_test_loss:.4f}")
print(f"    Test Accuracy: {bert_test_acc*100:.2f}%")
print(f"    Test F1 Score: {bert_test_f1:.4f}")
print(f"    Test AUC-ROC: {bert_test_auc:.4f}")



In [ ]:
# ============================================================================
# Part 3.3 Final Summary - All Models Including RNN + BERT
# ============================================================================

print("\n" + "="*80)
print("PART 3.3 FINAL SUMMARY - ALL MODELS")
print("="*80)
print(f"{'Model':<20} {'Val Acc':<12} {'Val F1':<12} {'Val AUC':<12} {'Test Acc':<12} {'Test F1':<12} {'Test AUC':<12}")
print("-" * 100)
print(f"{'biLSTM':<20} {bilstm_history['best_val_acc']*100:<12.2f}% {bilstm_history['best_val_f1']:<12.4f} {bilstm_history['best_val_auc']:<12.4f} {bilstm_test_acc*100:<12.2f}% {bilstm_test_f1:<12.4f} {bilstm_test_auc:<12.4f}")
print(f"{'biGRU':<20} {bigru_history['best_val_acc']*100:<12.2f}% {bigru_history['best_val_f1']:<12.4f} {bigru_history['best_val_auc']:<12.4f} {bigru_test_acc*100:<12.2f}% {bigru_test_f1:<12.4f} {bigru_test_auc:<12.4f}")
print(f"{'CNN':<20} {cnn_history['best_val_acc']*100:<12.2f}% {cnn_history['best_val_f1']:<12.4f} {cnn_history['best_val_auc']:<12.4f} {cnn_test_acc*100:<12.2f}% {cnn_test_f1:<12.4f} {cnn_test_auc:<12.4f}")
print(f"{'RNN+Attention':<20} {rnn_attn_history['best_val_acc']*100:<12.2f}% {rnn_attn_history['best_val_f1']:<12.4f} {rnn_attn_history['best_val_auc']:<12.4f} {rnn_attn_test_acc*100:<12.2f}% {rnn_attn_test_f1:<12.4f} {rnn_attn_test_auc:<12.4f}")
print(f"{'CNN+BiLSTM+Attn':<20} {hybrid_history['best_val_acc']*100:<12.2f}% {hybrid_history['best_val_f1']:<12.4f} {hybrid_history['best_val_auc']:<12.4f} {hybrid_test_acc*100:<12.2f}% {hybrid_test_f1:<12.4f} {hybrid_test_auc:<12.4f}")
print(f"{'RNN+BERT':<20} {bert_history['best_val_acc']*100:<12.2f}% {bert_history['best_val_f1']:<12.4f} {bert_history['best_val_auc']:<12.4f} {bert_test_acc*100:<12.2f}% {bert_test_f1:<12.4f} {bert_test_auc:<12.4f}")
print("="*100)

# Find best model based on test accuracy
all_models_final = {
    'biLSTM': {
        'val_acc': bilstm_history['best_val_acc'],
        'val_f1': bilstm_history['best_val_f1'],
        'val_auc': bilstm_history['best_val_auc'],
        'test_acc': bilstm_test_acc,
        'test_f1': bilstm_test_f1,
        'test_auc': bilstm_test_auc
    },
    'biGRU': {
        'val_acc': bigru_history['best_val_acc'],
        'val_f1': bigru_history['best_val_f1'],
        'val_auc': bigru_history['best_val_auc'],
        'test_acc': bigru_test_acc,
        'test_f1': bigru_test_f1,
        'test_auc': bigru_test_auc
    },
    'CNN': {
        'val_acc': cnn_history['best_val_acc'],
        'val_f1': cnn_history['best_val_f1'],
        'val_auc': cnn_history['best_val_auc'],
        'test_acc': cnn_test_acc,
        'test_f1': cnn_test_f1,
        'test_auc': cnn_test_auc
    },
    'RNN+Attention': {
        'val_acc': rnn_attn_history['best_val_acc'],
        'val_f1': rnn_attn_history['best_val_f1'],
        'val_auc': rnn_attn_history['best_val_auc'],
        'test_acc': rnn_attn_test_acc,
        'test_f1': rnn_attn_test_f1,
        'test_auc': rnn_attn_test_auc
    },
    'CNN+BiLSTM+Attention': {
        'val_acc': hybrid_history['best_val_acc'],
        'val_f1': hybrid_history['best_val_f1'],
        'val_auc': hybrid_history['best_val_auc'],
        'test_acc': hybrid_test_acc,
        'test_f1': hybrid_test_f1,
        'test_auc': hybrid_test_auc
    },
    'RNN+BERT': {
        'val_acc': bert_history['best_val_acc'],
        'val_f1': bert_history['best_val_f1'],
        'val_auc': bert_history['best_val_auc'],
        'test_acc': bert_test_acc,
        'test_f1': bert_test_f1,
        'test_auc': bert_test_auc
    }
}

best_model_test_acc = max(all_models_final.items(), key=lambda x: x[1]['test_acc'])
best_model_val_acc = max(all_models_final.items(), key=lambda x: x[1]['val_acc'])

print(f"\n>>> Best Model by Test Accuracy: {best_model_test_acc[0]}")
print(f"    Test Accuracy: {best_model_test_acc[1]['test_acc']*100:.2f}%")
print(f"    Test F1: {best_model_test_acc[1]['test_f1']:.4f}")
print(f"    Test AUC-ROC: {best_model_test_acc[1]['test_auc']:.4f}")

print(f"\n>>> Best Model by Validation Accuracy: {best_model_val_acc[0]}")
print(f"    Validation Accuracy: {best_model_val_acc[1]['val_acc']*100:.2f}%")
print(f"    Validation F1: {best_model_val_acc[1]['val_f1']:.4f}")
print(f"    Validation AUC-ROC: {best_model_val_acc[1]['val_auc']:.4f}")

print("\n>>> Part 3.3 Complete!")
print("    - RNN + Attention: Additive attention mechanism with bidirectional LSTM")
print("    - CNN + BiLSTM + Attention: Hybrid model combining CNN, BiLSTM, and attention")
print("    - RNN + BERT: Pretrained BERT embeddings with bidirectional LSTM and attention")
print("    - All models compared above with validation and test metrics")
print("    - All metrics (Accuracy, F1, AUC-ROC) reported for validation and test sets")



In [ ]:
# ============================================================================
# Plot Training Curves for RNN + BERT
# ============================================================================

print("\n" + "="*80)
print("PLOTTING TRAINING CURVES - RNN + BERT")
print("="*80)

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training Loss Curves (All Models)
axes[0, 0].plot(bilstm_history['train_losses'], label='biLSTM Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(bigru_history['train_losses'], label='biGRU Train Loss', marker='s', linewidth=2)
axes[0, 0].plot(cnn_history['train_losses'], label='CNN Train Loss', marker='^', linewidth=2)
axes[0, 0].plot(rnn_attn_history['train_losses'], label='RNN+Attention Train Loss', marker='d', linewidth=2)
axes[0, 0].plot(hybrid_history['train_losses'], label='CNN+BiLSTM+Attn Train Loss', marker='*', linewidth=2)
axes[0, 0].plot(bert_history['train_losses'], label='RNN+BERT Train Loss', marker='x', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Training Loss', fontsize=12)
axes[0, 0].set_title('Training Loss Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Accuracy Curves (All Models)
axes[0, 1].plot([acc*100 for acc in bilstm_history['val_accs']], label='biLSTM Val Acc', marker='o', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bigru_history['val_accs']], label='biGRU Val Acc', marker='s', linewidth=2)
axes[0, 1].plot([acc*100 for acc in cnn_history['val_accs']], label='CNN Val Acc', marker='^', linewidth=2)
axes[0, 1].plot([acc*100 for acc in rnn_attn_history['val_accs']], label='RNN+Attention Val Acc', marker='d', linewidth=2)
axes[0, 1].plot([acc*100 for acc in hybrid_history['val_accs']], label='CNN+BiLSTM+Attn Val Acc', marker='*', linewidth=2)
axes[0, 1].plot([acc*100 for acc in bert_history['val_accs']], label='RNN+BERT Val Acc', marker='x', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Validation Accuracy Curves (All Models)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: RNN+BERT Training and Validation Loss
axes[1, 0].plot(bert_history['train_losses'], label='RNN+BERT Train Loss', marker='o', linewidth=2)
axes[1, 0].plot(bert_history['val_losses'], label='RNN+BERT Val Loss', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('RNN+BERT: Training and Validation Loss', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: RNN+BERT Validation Accuracy
axes[1, 1].plot([acc*100 for acc in bert_history['val_accs']], label='RNN+BERT Val Acc', marker='o', linewidth=2, color='brown')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Validation Accuracy (%)', fontsize=12)
axes[1, 1].set_title('RNN+BERT: Validation Accuracy Curve', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_rnn_bert_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_3_rnn_bert_training_curves.png'")
plt.show()  # Display the plot in notebook

# Also create separate plots for Part 3.3 requirement (training loss and validation accuracy)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Curve
ax1.plot(bert_history['train_losses'], label='RNN+BERT Train Loss', marker='o', linewidth=2, color='blue')
ax1.plot(bert_history['val_losses'], label='RNN+BERT Val Loss', marker='s', linewidth=2, color='red')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss Curve (RNN+BERT)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in bert_history['val_accs']], label='RNN+BERT Val Acc', marker='o', linewidth=2, color='brown')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (RNN+BERT)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_3_bert_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved Part 3.3 BERT curves to 'part3_3_bert_training_curves.png'")
plt.show()  # Display the plot in notebook

print("\n>>> Training curves plotted and saved for RNN + BERT model")



In [ ]:
# c) 

# 1: (hybrid model) rnn + attention layer
# 2: (hybrid model) cnn + bilstm + attention layer
# 3: (hybrid model) bigru + pretrained transformer (berto)

In [ ]:
# ============================================================================
# PART 3.4: Targeted Improvement for Weak Topics
# Strategy: Data Augmentation, Positional Embeddings
# ============================================================================

print("\n" + "="*80)
print("PART 3.4: TARGETED IMPROVEMENT FOR WEAK TOPICS")
print("="*80)
print("\nStrategies:")
print("  1. Data Augmentation for imbalanced classes (especially ABBR)")
print("  2. Positional Embeddings in attention layer")
print("="*80)

# Import required libraries for augmentation
import nltk
from nltk.corpus import wordnet
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
except:
    pass


In [ ]:
# Load FastText model
print("\n    Loading FastText model...")
fatter_fasttext_bin = load_facebook_model('crawl-300d-2M-subword/crawl-300d-2M-subword.bin')
print("    ✓ FastText model loaded")

In [ ]:
# ============================================================================
# Step 1: Prepare Enhanced Embeddings with <mask> Token
# ============================================================================

print("\n>>> Step 1: Preparing Enhanced Embeddings with <mask> Token...")

# TEXT = data.Field ( tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True )

# # For multi - class classification labels
# LABEL = data.LabelField ()

# SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)

# # Load the TREC dataset
# # Train / Validation / Test split
# train_data, test_data = datasets.TREC.splits( TEXT, LABEL, fine_grained = False )

# train_data, validation_data = train_data.split(
#     split_ratio=0.8,
#     stratified=True,
#     strata_field='label'
# )
# print(vars(train_data.examples[0]))

TEXT_P34 = data.Field()
LABEL_P34 = data.LabelField()

train_data_p34 = data.Dataset(train_data.examples, train_data.fields)
TEXT_P34 = copy.deepcopy(TEXT)
LABEL_P34 = copy.deepcopy(LABEL)

# Add <mask> token to vocabulary
# mask_token = '<mask>'
# if mask_token not in TEXT_P34.vocab.stoi:
#     mask_index = len(TEXT_P34.vocab.stoi)
#     TEXT_P34.vocab.stoi[mask_token] = mask_index
#     TEXT_P34.vocab.itos.append(mask_token)
#     print(f"    ✓ Added <mask> token at index {mask_index}")
# else:
#     mask_index = TEXT_P34.vocab.stoi[mask_token]
#     print(f"    ✓ <mask> token already exists at index {mask_index}")

#TODO: Add loading of fasttext back here

pretrained_embeddings_p34 = None
embedding_dim_p34 = fatter_fasttext_bin.wv.vector_size

# Build embedding matrix with <mask> token
num_tokens_p34 = len(TEXT_P34.vocab.stoi)
emb_matrix_p34 = np.zeros((num_tokens_p34, embedding_dim_p34), dtype=np.float32)

pad_tok = TEXT_P34.pad_token
unk_tok = TEXT_P34.unk_token
pad_index = TEXT_P34.vocab.stoi[pad_tok]
unk_index = TEXT_P34.vocab.stoi[unk_tok]

known_vecs_p34 = []
mask_candidates = []

print("    Processing vocabulary tokens...")
for idx, token in enumerate(TEXT_P34.vocab.itos):
    # if token in {pad_tok, unk_tok, mask_token}:
    if token in {pad_tok, unk_tok}:
        continue
    
    try:
        vec = fatter_fasttext_bin.wv[token]
        emb_matrix_p34[idx] = vec
        known_vecs_p34.append(vec)
        if len(mask_candidates) < 1000:
            mask_candidates.append(vec)
    except KeyError:
        try:
            vec = fatter_fasttext_bin.wv[token.lower()]
            emb_matrix_p34[idx] = vec
            known_vecs_p34.append(vec)
            if len(mask_candidates) < 1000:
                mask_candidates.append(vec)
        except KeyError:
            emb_matrix_p34[idx] = np.random.uniform(-0.05, 0.05, embedding_dim_p34).astype(np.float32)

# Initialize special tokens
if len(known_vecs_p34) > 0:
    unk_mean_p34 = np.mean(known_vecs_p34, axis=0)
    emb_matrix_p34[unk_index] = unk_mean_p34
    print(f"    ✓ <unk> initialized as mean of {len(known_vecs_p34)} vectors")

# if len(mask_candidates) > 0:
#     mask_mean_p34 = np.mean(mask_candidates, axis=0)
#     emb_matrix_p34[mask_index] = mask_mean_p34
#     print(f"    ✓ <mask> initialized as mean of {len(mask_candidates)} vectors")
# else:
#     emb_matrix_p34[mask_index] = emb_matrix_p34[unk_index]

emb_matrix_p34[pad_index] = np.zeros(embedding_dim_p34, dtype=np.float32)

# Create and save embedding layer
p34_embedding = torch.nn.Embedding(
    num_tokens_p34, 
    embedding_dim_p34, 
    padding_idx=pad_index
)
p34_embedding.weight.data.copy_(torch.from_numpy(emb_matrix_p34))
p34_embedding.weight.requires_grad = True

torch.save(p34_embedding, 'embedding_weights_part34_enhanced.pt')
pretrained_embeddings_p34 = p34_embedding.weight.data.clone()

print(f"    ✓ Embeddings saved (shape: {pretrained_embeddings_p34.shape})")
print(f"    ✓ Ready for Part 3.4 training")


In [ ]:
len(TEXT.vocab.stoi)

In [ ]:
# ============================================================================
# Step 2: Data Augmentation Functions for Imbalanced Classes
# ============================================================================

print("\n>>> Step 2: Implementing Data Augmentation Functions...")

def get_synonyms(word):
    """Get synonyms for a word using WordNet"""
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').lower()
            if synonym != word and synonym.isalpha():
                synonyms.add(synonym)
    return list(synonyms)

def synonym_replacement(tokens, n=1):
    """Replace n random words with their synonyms"""
    new_tokens = tokens.copy()
    words_to_replace = [i for i, word in enumerate(tokens) if word.isalpha() and len(word) > 2]
    
    if len(words_to_replace) == 0:
        return tokens
    
    num_replacements = min(n, len(words_to_replace))
    indices_to_replace = random.sample(words_to_replace, num_replacements)
    
    for idx in indices_to_replace:
        synonyms = get_synonyms(tokens[idx])
        if synonyms:
            new_tokens[idx] = random.choice(synonyms)
    
    return new_tokens

def random_insertion(tokens, n=1):
    """Randomly insert synonyms of n words"""
    new_tokens = tokens.copy()
    
    for _ in range(n):
        if len(new_tokens) == 0:
            break
        word = random.choice(new_tokens)
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            insert_pos = random.randint(0, len(new_tokens))
            new_tokens.insert(insert_pos, synonym)
    
    return new_tokens

def random_deletion(tokens, p=0.1):
    """Randomly delete words with probability p"""
    if len(tokens) == 1:
        return tokens
    
    new_tokens = []
    for token in tokens:
        if random.random() > p:
            new_tokens.append(token)
    
    if len(new_tokens) == 0:
        return tokens[:1]
    
    return new_tokens

def random_swap(tokens, n=1):
    """Randomly swap n pairs of words"""
    new_tokens = tokens.copy()
    
    for _ in range(n):
        if len(new_tokens) < 2:
            break
        idx1, idx2 = random.sample(range(len(new_tokens)), 2)
        new_tokens[idx1], new_tokens[idx2] = new_tokens[idx2], new_tokens[idx1]
    
    return new_tokens

def augment_text(text, augmentation_techniques=['synonym', 'insertion', 'deletion', 'swap'], 
                 num_augmentations=3):
    """Apply data augmentation to text"""
    augmented_texts = []
    
    for _ in range(num_augmentations):
        aug_text = text.copy()
        technique = random.choice(augmentation_techniques)
        
        if technique == 'synonym' and len(aug_text) > 0:
            aug_text = synonym_replacement(aug_text, n=random.randint(1, 2))
        elif technique == 'insertion' and len(aug_text) > 0:
            aug_text = random_insertion(aug_text, n=random.randint(1, 2))
        elif technique == 'deletion' and len(aug_text) > 1:
            aug_text = random_deletion(aug_text, p=0.1)
        elif technique == 'swap' and len(aug_text) > 1:
            aug_text = random_swap(aug_text, n=1)
        
        augmented_texts.append(aug_text)
    
    return augmented_texts

print("    ✓ Data augmentation functions ready")


In [ ]:


# ============================================================================
# Step 3: Apply Data Augmentation for Imbalanced Classes
# ============================================================================

print("\n>>> Step 3: Applying Data Augmentation for Imbalanced Classes...")

# Count current label distribution
label_counts_p34 = Counter([ex.label for ex in train_data.examples])
print(f"\nOriginal label distribution:")
for label, count in sorted(label_counts_p34.items()):
    print(f"  {label}: {count} samples ({count/len(train_data.examples)*100:.2f}%)")

# Target counts: Augment ABBR significantly, keep others similar
target_counts_p34 = {
    'ABBR': 500,   # Augment from 69 to 500 (~7x)
    'DESC': 930,   # Keep original
    'ENTY': 1000,  # Keep original
    'HUM': 978,    # Keep original
    'LOC': 668,    # Keep original
    'NUM': 717     # Keep original
}

# Create augmented examples
augmented_examples = list(train_data.examples)  # Start with all original examples

for label in label_counts_p34.keys():
    current_count = label_counts_p34[label]
    target_count = target_counts_p34[label]
    
    if current_count < target_count:
        label_examples = [ex for ex in train_data.examples if ex.label == label]
        num_augmentations_needed = target_count - current_count
        
        print(f"\n  Augmenting {label}: {current_count} -> {target_count} samples")
        print(f"    Generating {num_augmentations_needed} additional samples...")
        
        augmented_count = 0
        while augmented_count < num_augmentations_needed:
            original_ex = random.choice(label_examples)
            aug_texts = augment_text(original_ex.text, num_augmentations=1)
            
            for aug_text in aug_texts:
                if augmented_count >= num_augmentations_needed:
                    break
                
                new_ex = data.Example.fromlist([aug_text, label], 
                                               fields=[('text', TEXT_P34), ('label', LABEL_P34)])
                augmented_examples.append(new_ex)
                augmented_count += 1
        
        print(f"    ✓ Generated {augmented_count} augmented samples")

# Create augmented dataset with proper field structure
augmented_train_data = data.Dataset(augmented_examples, fields=[('text', TEXT_P34), ('label', LABEL_P34)])

# Verify augmented distribution
new_label_counts = Counter([ex.label for ex in augmented_examples])
print(f"\nAugmented label distribution:")
for label, count in sorted(new_label_counts.items()):
    print(f"  {label}: {count} samples ({count/len(augmented_examples)*100:.2f}%)")

print(f"\n  Total samples: {len(train_data.examples)} -> {len(augmented_examples)}")
print(f"  ✓ Data augmentation complete")

In [ ]:
augmented_train_data.examples[0].label

In [ ]:
LABEL_P34.vocab.stoi

In [ ]:
abbr_aug_ex = [ex for ex in augmented_train_data if ex.label == "ABBR"]
abbr_ex = [ex for ex in train_data if ex.label == "ABBR"]

In [ ]:
[(ex.text, ex.label) for ex in augmented_train_data if ex.label not in ['ABBR','DESC','ENTY','HUM','LOC','NUM']]

In [ ]:
count = 0
for ex in abbr_aug_ex:
    if ex not in abbr_ex:
        print(ex.text)
        print(ex.label)
        count += 1

print(count)

In [ ]:
# Count how many samples per label in the train set
label_counts_p34 = Counter([ex.label for ex in augmented_train_data.examples])
total_examples_p34 = len(augmented_train_data)

print("\nLabel distribution in training set:")
for label, count in sorted(label_counts_p34.items()):
    percentage = (count / total_examples_p34) * 100
    print(f"- {label}: {count} samples ({percentage:.2f}%)")

In [ ]:
# ============================================================================
# Step 5: RNN Model with Positional Embeddings in Attention Layer
# ============================================================================

print("\n>>> Step 5: Creating RNN Model with Positional Embeddings...")

class RNNWithPositionalAttentionClassifier(nn.Module):
    """
    RNN with Attention and Positional Embeddings
    Adds positional information to help the model understand word order
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=2, dropout=0.5, padding_idx=0, pretrained_embeddings=None,
                 attention_dim=None, max_seq_length=100, use_positional_emb=True):
        super(RNNWithPositionalAttentionClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.attention_dim = attention_dim if attention_dim else hidden_dim
        self.embedding_dim = embedding_dim
        self.max_seq_length = max_seq_length
        self.use_positional_emb = use_positional_emb
        
        # Word Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            self.embedding.weight.requires_grad = True
        
        # Positional Embedding layer (learnable)
        if self.use_positional_emb:
            self.positional_embedding = nn.Embedding(max_seq_length, embedding_dim)
            # Initialize with sinusoidal pattern
            self._init_positional_embeddings()
        
        # Bidirectional LSTM layer
        self.rnn = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if n_layers > 1 else 0
        )
        
        # Attention Mechanism with positional awareness
        self.attention_linear1 = nn.Linear(hidden_dim * 2, self.attention_dim)
        self.attention_linear2 = nn.Linear(self.attention_dim, 1)
        self.tanh = nn.Tanh()
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def _init_positional_embeddings(self):
        """Initialize positional embeddings with sinusoidal pattern"""
        pe = torch.zeros(self.max_seq_length, self.embedding_dim)
        position = torch.arange(0, self.max_seq_length).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, self.embedding_dim, 2).float() * 
                            -(np.log(10000.0) / self.embedding_dim))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Set as parameter data (no gradient for positional embeddings initially, but can be learned)
        with torch.no_grad():
            self.positional_embedding.weight.data = pe
    
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        batch_size, seq_len = text.size()
        
        # Word Embeddings
        word_embeddings = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Positional Embeddings
        if self.use_positional_emb:
            positions = torch.arange(0, seq_len, device=text.device).unsqueeze(0).expand(batch_size, -1)
            positions = torch.clamp(positions, 0, self.max_seq_length - 1)
            pos_embeddings = self.positional_embedding(positions)  # [batch_size, seq_len, embedding_dim]
            
            # Combine word and positional embeddings
            embedded = word_embeddings + pos_embeddings
        else:
            embedded = word_embeddings
        
        # Pack sequences for efficient RNN processing
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        # Pass through bidirectional LSTM
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        # Unpack the sequences
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True
        )
        # output: [batch_size, seq_len, hidden_dim * 2]
        
        # Apply Attention Mechanism with positional information
        attention_scores = self.attention_linear1(output)  # [batch_size, seq_len, attention_dim]
        attention_scores = self.tanh(attention_scores)
        attention_scores = self.attention_linear2(attention_scores).squeeze(2)  # [batch_size, seq_len]
        
        # Mask padding positions
        mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths.unsqueeze(1)
        attention_scores = attention_scores.masked_fill(~mask, float('-inf'))
        
        # Apply softmax to get attention weights
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(2)  # [batch_size, seq_len, 1]
        
        # Compute weighted sum of RNN outputs
        context_vector = torch.sum(attention_weights * output, dim=1)  # [batch_size, hidden_dim * 2]
        
        # Apply dropout
        context_vector = self.dropout(context_vector)
        
        # Pass through fully connected layer
        output = self.fc(context_vector)  # [batch_size, output_dim]
        
        return output

print("    ✓ RNNWithPositionalAttentionClassifier model created")
print("    Features:")
print("      - Word embeddings (FastText pretrained)")
print("      - Positional embeddings (sinusoidal pattern)")
print("      - Bidirectional LSTM")
print("      - Attention mechanism with positional awareness")


In [ ]:
# ============================================================================
# Step 6: Create Iterators
# ============================================================================

print("\n>>> Step 6: Creating Iterators...")

P34_BATCH_SIZE = 64

# Create iterators
train_iter_p34 = data.BucketIterator(
    augmented_train_data,
    batch_size=P34_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=True,
    device=device
)

# Validation and test iterators
val_iter_p34 = data.BucketIterator(
    validation_data,
    batch_size=P34_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

test_iter_p34 = data.BucketIterator(
    test_data,
    batch_size=P34_BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    shuffle=False,
    device=device
)

print(f"    ✓ Training iterator: {len(augmented_train_data)} samples")
print(f"    ✓ Validation iterator: {len(validation_data)} samples")
print(f"    ✓ Test iterator: {len(test_data)} samples")
print(f"    ✓ Batch size: {P34_BATCH_SIZE}")


In [ ]:
TEXT_P34.vocab.stoi['<mask>']

In [ ]:
# ============================================================================
# Step 7: Train Part 3.4 Model
# ============================================================================

print("\n>>> Step 7: Training Part 3.4 Enhanced Model...")

# Model hyperparameters
P34_EMBEDDING_DIM = embedding_dim_p34
P34_HIDDEN_DIM = 256
P34_N_LAYERS = 2
P34_DROPOUT = 0.5
P34_ATTENTION_DIM = 256
P34_MAX_SEQ_LENGTH = 100
P34_LEARNING_RATE = 0.001
P34_N_EPOCHS = 50
P34_PATIENCE = 10
P34_NUM_CLASSES = len(LABEL_P34.vocab)

# Create model
p34_model = RNNWithPositionalAttentionClassifier(
    vocab_size=len(TEXT_P34.vocab.stoi),
    embedding_dim=P34_EMBEDDING_DIM,
    hidden_dim=P34_HIDDEN_DIM,
    output_dim=P34_NUM_CLASSES,
    n_layers=P34_N_LAYERS,
    dropout=P34_DROPOUT,
    padding_idx=TEXT_P34.vocab.stoi[TEXT_P34.pad_token],
    pretrained_embeddings=pretrained_embeddings_p34,
    attention_dim=P34_ATTENTION_DIM,
    max_seq_length=P34_MAX_SEQ_LENGTH,
    use_positional_emb=True
).to(device)

print(f"\n>>> Part 3.4 Model Configuration:")
print(f"    Embedding Dim: {P34_EMBEDDING_DIM}")
print(f"    Hidden Dim: {P34_HIDDEN_DIM}")
print(f"    Layers: {P34_N_LAYERS}")
print(f"    Dropout: {P34_DROPOUT}")
print(f"    Attention Dim: {P34_ATTENTION_DIM}")
print(f"    Max Seq Length: {P34_MAX_SEQ_LENGTH}")
print(f"    Positional Embeddings: Enabled")
print(f"    Parameters: {sum(p.numel() for p in p34_model.parameters()):,}")

# Loss function and optimizer
p34_criterion = nn.CrossEntropyLoss()
p34_optimizer = optim.Adam(p34_model.parameters(), lr=P34_LEARNING_RATE)

# Training function (using existing train_model_with_history if available, otherwise create one)
def process_batch_p34(batch):
    """Process batch for Part 3.4"""
    text, text_lengths = batch.text
    labels = batch.label
    return text, text_lengths, labels

# Train the model
print("\n>>> Starting training...")
p34_model.train()
best_val_acc = 0.0
best_val_f1 = 0.0
best_val_auc = 0.0
patience_counter = 0
best_model_state = None

train_losses_p34 = []
val_losses_p34 = []
val_accs_p34 = []
val_f1s_p34 = []
val_aucs_p34 = []

for epoch in range(P34_N_EPOCHS):
    start_time = time.time()
    
    # Training phase
    p34_model.train()
    train_loss = 0
    train_preds = []
    train_labels = []
    
    for batch in train_iter_p34:
        text, text_lengths, labels = process_batch_p34(batch)
        p34_optimizer.zero_grad()
        predictions = p34_model(text, text_lengths)
        loss = p34_criterion(predictions, labels)
        loss.backward()
        p34_optimizer.step()
        
        train_loss += loss.item()
        preds = torch.argmax(predictions, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
    
    avg_train_loss = train_loss / len(train_iter_p34)
    train_acc = accuracy_score(train_labels, train_preds)
    
    # Validation phase
    p34_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    val_probs = []
    
    with torch.no_grad():
        for batch in val_iter_p34:
            text, text_lengths, labels = process_batch_p34(batch)
            predictions = p34_model(text, text_lengths)
            loss = p34_criterion(predictions, labels)
            val_loss += loss.item()
            
            probs = torch.softmax(predictions, dim=1)
            preds = torch.argmax(predictions, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_probs.extend(probs.cpu().numpy())
    
    avg_val_loss = val_loss / len(val_iter_p34)
    val_acc = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
    # Calculate AUC-ROC
    try:
        val_probs_array = np.array(val_probs)
        val_labels_bin = label_binarize(val_labels, classes=range(P34_NUM_CLASSES))
        val_auc = roc_auc_score(val_labels_bin, val_probs_array, average='weighted', multi_class='ovr')
    except:
        val_auc = 0.0
    
    # Store history
    train_losses_p34.append(avg_train_loss)
    val_losses_p34.append(avg_val_loss)
    val_accs_p34.append(val_acc)
    val_f1s_p34.append(val_f1)
    val_aucs_p34.append(val_auc)
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_val_f1 = val_f1
        best_val_auc = val_auc
        patience_counter = 0
        best_model_state = p34_model.state_dict().copy()
    else:
        patience_counter += 1
    
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    print(f'Epoch: {epoch+1:02}/{P34_N_EPOCHS} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {avg_train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal Loss: {avg_val_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}')
    
    if patience_counter >= P34_PATIENCE:
        print(f'\t>>> Early stopping at epoch {epoch+1}, best val acc: {best_val_acc*100:.2f}%')
        break

# Load best model state
if best_model_state is not None:
    p34_model.load_state_dict(best_model_state)

# Save model
torch.save(p34_model.state_dict(), 'weights/part34_enhanced_best.pt')
print(f"\n>>> Part 3.4 model saved to 'weights/part34_enhanced_best.pt'")

p34_history = {
    'train_losses': train_losses_p34,
    'val_losses': val_losses_p34,
    'val_accs': val_accs_p34,
    'val_f1s': val_f1s_p34,
    'val_aucs': val_aucs_p34,
    'best_val_acc': best_val_acc,
    'best_val_f1': best_val_f1,
    'best_val_auc': best_val_auc
}


In [ ]:
# ============================================================================
# Step 8: Evaluate Part 3.4 Model on Test Set
# ============================================================================

print("\n>>> Step 8: Evaluating Part 3.4 Model on Test Set...")

p34_model.eval()
test_loss_p34 = 0
test_preds_p34 = []
test_labels_p34 = []
test_probs_p34 = []

with torch.no_grad():
    for batch in test_iter_p34:
        text, text_lengths, labels = process_batch_p34(batch)
        predictions = p34_model(text, text_lengths)
        loss = p34_criterion(predictions, labels)
        test_loss_p34 += loss.item()
        
        probs = torch.softmax(predictions, dim=1)
        preds = torch.argmax(predictions, dim=1)
        test_preds_p34.extend(preds.cpu().numpy())
        test_labels_p34.extend(labels.cpu().numpy())
        test_probs_p34.extend(probs.cpu().numpy())

test_loss_p34 = test_loss_p34 / len(test_iter_p34)
test_acc_p34 = accuracy_score(test_labels_p34, test_preds_p34)
test_f1_p34 = f1_score(test_labels_p34, test_preds_p34, average='weighted')

try:
    test_probs_array_p34 = np.array(test_probs_p34)
    test_labels_bin_p34 = label_binarize(test_labels_p34, classes=range(P34_NUM_CLASSES))
    test_auc_p34 = roc_auc_score(test_labels_bin_p34, test_probs_array_p34, average='weighted', multi_class='ovr')
except:
    test_auc_p34 = 0.0

print(f"\n>>> Part 3.4 Test Set Results:")
print(f"    Test Loss: {test_loss_p34:.4f}")
print(f"    Test Accuracy: {test_acc_p34*100:.2f}%")
print(f"    Test F1 Score: {test_f1_p34:.4f}")
print(f"    Test AUC-ROC: {test_auc_p34:.4f}")


In [ ]:
# ============================================================================
# Step 9: Topic-wise Accuracy Evaluation and Comparison
# ============================================================================

print("\n>>> Step 9: Topic-wise Accuracy Evaluation...")

def evaluate_per_topic_p34(model, iterator, device):
    """Evaluate model performance per topic category"""
    model.eval()
    
    topic_correct = defaultdict(int)
    topic_total = defaultdict(int)
    
    idx_to_label = LABEL_P34.vocab.itos
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths, labels = process_batch_p34(batch)
            predictions = model(text, text_lengths)
            preds = torch.argmax(predictions, dim=1)
            
            for pred, label in zip(preds.cpu().numpy(), labels.cpu().numpy()):
                topic_name = idx_to_label[label]
                topic_total[topic_name] += 1
                if pred == label:
                    topic_correct[topic_name] += 1
    
    # Calculate accuracy per topic
    topic_accuracies = {}
    for topic in sorted(topic_total.keys()):
        acc = topic_correct[topic] / topic_total[topic] if topic_total[topic] > 0 else 0
        topic_accuracies[topic] = acc
        print(f'  {topic}: {topic_correct[topic]}/{topic_total[topic]} = {acc*100:.2f}%')
    
    return topic_accuracies

print("\n>>> Part 3.4 Enhanced Model - Topic-wise Accuracy:")
p34_topic_acc = evaluate_per_topic_p34(p34_model, test_iter_p34, device)

# Note: To compare with Part 2(e), you would need to load and evaluate the baseline model
# For now, we'll display Part 3.4 results
print("\n" + "="*80)
print("PART 3.4 TOPIC-WISE ACCURACY SUMMARY")
print("="*80)
print(f"{'Topic':<10} {'Accuracy':<12} {'Correct':<10} {'Total':<10}")
print("-" * 50)
for topic in sorted(p34_topic_acc.keys()):
    # Get counts from evaluation
    # (We'd need to modify evaluate_per_topic_p34 to return counts, or re-evaluate)
    acc = p34_topic_acc[topic]
    print(f"{topic:<10} {acc*100:<12.2f}%")

print("\n>>> Note: Compare these results with Part 2(e) baseline for improvement analysis")


In [ ]:
# ============================================================================
# Step 10: Plot Training Curves for Part 3.4
# ============================================================================

print("\n>>> Step 10: Plotting Training Curves...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Curve
ax1.plot(p34_history['train_losses'], label='Part 3.4 Train Loss', marker='o', linewidth=2, color='blue')
ax1.plot(p34_history['val_losses'], label='Part 3.4 Val Loss', marker='s', linewidth=2, color='red')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss Curve (Part 3.4)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Validation Accuracy Curve
ax2.plot([acc*100 for acc in p34_history['val_accs']], label='Part 3.4 Val Acc', marker='o', linewidth=2, color='purple')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Validation Accuracy (%)', fontsize=12)
ax2.set_title('Validation Accuracy Curve (Part 3.4)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('part3_4_training_curves.png', dpi=300, bbox_inches='tight')
print(f"    Saved training curves to 'part3_4_training_curves.png'")
plt.show()

print("\n>>> Training curves plotted and saved")


In [ ]:
# ============================================================================
# PART 3.4 SUMMARY
# ============================================================================

print("\n" + "="*80)
print("PART 3.4: TARGETED IMPROVEMENT FOR WEAK TOPICS - SUMMARY")
print("="*80)

print("\n>>> Strategies Implemented:")
print("  1. Data Augmentation for Imbalanced Classes")
print("     - ABBR: 69 -> 500 samples (~7x augmentation)")
print("     - Used synonym replacement, random insertion, deletion, and swap")
print("     - Other classes maintained original size")
print("\n  2. Positional Embeddings in Attention Layer")
print("     - Sinusoidal positional embeddings added to word embeddings")
print("     - Helps model understand word order and sequence structure")
print("     - Integrated into attention mechanism")

print("\n>>> Model Configuration:")
print(f"  - Embedding Dim: {P34_EMBEDDING_DIM}")
print(f"  - Hidden Dim: {P34_HIDDEN_DIM}")
print(f"  - Layers: {P34_N_LAYERS} (bidirectional LSTM)")
print(f"  - Dropout: {P34_DROPOUT}")
print(f"  - Attention Dim: {P34_ATTENTION_DIM}")
print(f"  - Max Seq Length: {P34_MAX_SEQ_LENGTH}")
print(f"  - Learning Rate: {P34_LEARNING_RATE}")
print(f"  - Batch Size: {P34_BATCH_SIZE}")

print("\n>>> Results:")
print(f"  - Best Val Accuracy: {p34_history['best_val_acc']*100:.2f}%")
print(f"  - Best Val F1: {p34_history['best_val_f1']:.4f}")
print(f"  - Best Val AUC: {p34_history['best_val_auc']:.4f}")
print(f"  - Test Accuracy: {test_acc_p34*100:.2f}%")
print(f"  - Test F1: {test_f1_p34:.4f}")
print(f"  - Test AUC: {test_auc_p34:.4f}")

print("\n>>> Topic-wise Accuracy (Test Set):")
for topic in sorted(p34_topic_acc.keys()):
    print(f"  - {topic}: {p34_topic_acc[topic]*100:.2f}%")

print("\n>>> Expected Improvements:")
print("  - ABBR accuracy should improve significantly due to:")
print("    * 7x data augmentation (69 -> 500 samples)")
print("    * Positional embeddings helping with question structure")
print("\n  - Other topics should maintain or slightly improve performance")

print("\n>>> Files Saved:")
print("  - Model: weights/part34_enhanced_best.pt")
print("  - Embeddings: embedding_weights_part34_enhanced.pt")
print("  - Training Curves: part3_4_training_curves.png")

print("\n" + "="*80)
print("PART 3.4 COMPLETE")
print("="*80)
